In [ ]:
import pandas as pd
import numpy as np

from astropy.coordinates import SkyCoord, angular_separation, SkyOffsetFrame
import astropy.units as u
from astropy.modeling import models, fitting
from astropy.table import Table, vstack
from astropy.io import fits
from astropy.wcs import WCS
from astropy.nddata.utils import Cutout2D
from astropy.visualization import simple_norm
from reproject import reproject_exact, reproject_interp
#from bayestar import SFH
import json
import glob
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoLocator, AutoMinorLocator)
from gloess import gloess
from catalog_filter import box, ellipse
from cmdtools import gen_CMD, gen_CMD_xcut, gen_CMD_ycut, running_avg
from astropy.stats import SigmaClip, sigma_clipped_stats, gaussian_fwhm_to_sigma
from astropy.stats.biweight import biweight_location, biweight_midvariance,biweight_scale
from pydol import photometry
from pathlib import Path
import subprocess
from scipy.interpolate import griddata
from astropy.visualization.wcsaxes import SphericalCircle
from astropy.visualization.wcsaxes import add_beam, add_scalebar
import matplotlib.font_manager as fm

In [ ]:
script_dir = str(Path(photometry.__file__).parent.joinpath('scripts'))

In [ ]:
plt.rcParams.update({
    
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})


In [ ]:
from numpy import append, array, exp, \
                linspace, loadtxt, log10, pi, meshgrid, \
                savetxt, sqrt, where, ones, percentile,trapz, all
from scipy import special
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import time, glob, os, sys
import multiprocessing as mp
from pathlib import Path

import stan

code = """

    functions{
        real P(int N1, int N2, vector v, matrix M) {
            vector[N1] Mj;
            vector[N1] ln_Mj;

            Mj= M*v;
            for (j in 1:N1){
                if (Mj[j]<=0.)
                    Mj[j] = 1.;
            }
            ln_Mj = log(Mj);
            return sum(ln_Mj);
        }
    }

    data {
        int<lower=0> Nj; // number of data
        int<lower=0> Ni; // number of isochrones
        matrix[Nj,Ni] Pij; // Probability matrix
        matrix[Nj,Ni] Cij; // Normalization matrix
    }

    parameters {
        simplex[Ni] a;
    }

    model {
        target += dirichlet_lpdf(a | rep_vector(1., Ni));
        target += P(Nj,Ni,a,Pij);
        target += -1.*P(Nj,Ni,a,Cij);
    }

    """

class Base():

    def Normal_MGk(self, fw_dat,fw_err,Iso_sig):    # Error model apparent maginute
        sig2 = fw_err*fw_err + Iso_sig*Iso_sig          # And normal Isochrone 
        
        return  lambda fw_iso : exp( -0.5*(fw_dat-fw_iso)**2 / sig2 ) / sqrt(2.*pi*sig2)

    def Phi_MGk(self,fwj2, sig_fwj2, fwklim, sig_i2):
        b = sig_i2*sig_i2 + sig_fwj2*sig_fwj2
        b1 = sig_i2*sig_i2/b
        b2 = sig_fwj2*sig_fwj2/b
        b3 = sig_i2*sig_i2/sqrt(b)
        return  lambda fw_i2 : special.ndtr( ( fwklim - b1*fwj2 - b2*fw_i2 ) / b3 )

    def IMF_Krp(self, m, ml=0.1, mint=0.5, mu=350.,a1=1.3,a2=2.3):

        h2 = (mu**(1.-a2)-mint**(1.-a2))/(1.-a2)
        h1 = (mint**(1.-a1)-ml**(1.-a1))/(1.-a1)

        c1 = 1./(h1+h2*mint**(a2-a1))
        c2 = c1*mint**(a2-a1)

        c = ones(len(m))
        c[where(m < mint)] = c1
        c[where(m >= mint)] = c2

        a = ones(len(m))
        a[where(m < mint)] = -a1
        a[where(m >= mint)] = -a2
        imf = c*m**a

        return(imf)


    def P_ij_map(self, IDp):
        fw1_lim = self.fw_lims[0]
        fw2_lim = self.fw_lims[1]
        fw3_lim = self.fw_lims[2]

        sig_i = self.sig_fw[0]

        if not os.path.exists(self.out_dir + 'pij_cij_results'):
            os.mkdir(self.out_dir + 'pij_cij_results')

        filename_p = '%s_Pij_Data_LimMag%.2lf_%srows_%siso_IsoModel_sig%s_IMF_%s_Simple.txt' % \
            (IDp,fw2_lim,str(self.N_dat),str(self.NIso),str(sig_i).replace('.','p'), self.IMF)    ## Opening file
        fp = open(os.path.join(self.out_dir + "pij_cij_results",filename_p),'a')
        args = []

         ## Pij is calcutated row by row, i.e. fix j-th dat and run each i-th isochrone.

        for j in range(self.N_dat):                       
    #                    0   1     2    3     4         5        6      7       8        9     10
            args.append([j, self.dat, self.NIso, self.Iso, 
                         fw1_lim, fw2_lim, fw3_lim, self.N_dat, filename_p, sig_i, self.IMF])
        with mp.Pool(mp.cpu_count()-1) as p:         ## Pooling Pij rows using all the abailable CPUs (Parallel computation)
            results = p.map(self.P_ij_row_map, args)
            Pij_out=[]
            for [j,wr] in results:
                fp.write('{}'.format(' '.join(wr))+'\n')
                Pij_out.append(array(wr, dtype=float))
        fp.close()
        
        return([Pij_out, filename_p])

    def P_ij_row_map(self, args):

        j = args[0]
        dat = args[1]
        Niso = args[2]
        Iso = args[3]

        fw1_lim = args[4]
        fw2_lim = args[5]
        fw3_lim = args[6]

        Ndat = args[7]
        filename_p = args[8]
        sig_i = args[9]
        imf = args[10]
        
        P_fw1 = self.Normal_MGk(self.dat[2][j],self.dat[3][j],sig_i)
        P_fw2 = self.Normal_MGk(self.dat[4][j],self.dat[5][j],sig_i)
        P_fw3 = self.Normal_MGk(self.dat[6][j],self.dat[7][j],sig_i)

        Phi_fw1 = self.Phi_MGk(self.dat[2][j], self.dat[3][j], fw1_lim, sig_i)
        Phi_fw2 = self.Phi_MGk(self.dat[4][j], self.dat[5][j], fw2_lim, sig_i)
        Phi_fw3 = self.Phi_MGk(self.dat[6][j], self.dat[7][j], fw3_lim, sig_i)

        wr=[]
        for i in range(self.NIso):                    ## Isochrone loop

            if self.IMF == "Krp":
                imf_p = self.IMF_Krp(self.Iso[i][1])
            elif self.IMF == "Slp":
                print("Not available at the moment")
                #imf_p = self.IMF_Salp(Iso[i][1])
            else:
                # Default
                imf_p = self.IMF_Krp(self.Iso[i][1])

            Ps = P_fw1(self.Iso[i][2])*P_fw2(self.Iso[i][3])*P_fw3(self.Iso[i][4])
            
            Phis = Phi_fw1(self.Iso[i][2])*Phi_fw2(self.Iso[i][3])*Phi_fw3(self.Iso[i][4])
            delta_t = 10**self.Iso[i][6]-10**(self.Iso[i][6]-0.1)
            delta_tc = (10**self.Z_age_isos[:,1] - 10**(self.Z_age_isos[:,1]-0.1)).sum()
            Intg = imf_p*Ps*Phis*(delta_t/delta_tc)#*self.Iso[i][7]

            ## Interand
            p = trapz(Intg,self.Iso[i][1])

            wr.append(str(p))

        return ([j,wr])

    def C_ij_map(self, IDc):

        fw1_lim= self.fw_lims[0]
        fw2_lim= self.fw_lims[1]
        fw3_lim= self.fw_lims[2]
        
        sig_i = self.sig_fw[0]

        filename_c = '%s_Cij_Data_LimMag%.2lf_%srows_%siso_IsoModel_sig%s_IMF_%s_Simple.txt' % \
            (IDc,fw2_lim,str(self.N_dat),str(self.NIso),str(self.sig_fw[0]).replace('.','p'), self.IMF)
        fp = open(os.path.join(self.out_dir + "pij_cij_results",filename_c),'a')   ## output matrix
        args = []

        # Cij is calcutated row by row, i.e. fix j-th dat and run each i-th isochrone.

        for j in range(self.N_dat):    
            args.append([j, self.dat, self.NIso, self.Iso, fw1_lim, 
                         fw2_lim, fw3_lim, self.N_dat, filename_c, sig_i, self.IMF])

        with mp.Pool(mp.cpu_count()-1) as p:
            results = p.map(self.C_ij_row_map, args)
            Cij_out=[]
            for [j,wr] in results:
                fp.write('{}'.format(' '.join(wr))+'\n')
                Cij_out.append(array(wr, dtype=float))
        fp.close()
        
        return(Cij_out)

    def C_ij_row_map(self,args):

        j = args[0]
        dat = args[1]
        Niso = args[2]
        Iso = args[3]

        fw1_lim = args[4]
        fw2_lim = args[5]
        fw3_lim = args[6]

        Ndat = args[7]
        filename_c = args[8]
        sig_i = args[9]
        imf = args[10]

        phi_fw1 = self.Phi_MGk(dat[2][j], dat[3][j], fw1_lim, sig_i)
        phi_fw2 = self.Phi_MGk(dat[4][j], dat[5][j], fw2_lim, sig_i)
        phi_fw3 = self.Phi_MGk(dat[6][j], dat[7][j], fw2_lim, sig_i)

        wr = []
        for i in range(Niso):

            if imf == "Krp":
                imf_c = self.IMF_Krp(Iso[i][1])
            elif imf=="Slp":
                print("Not available at the moment")
                #imf_c = self.IMF_Salp(Iso[i][1])
            else:
                imf_c = self.IMF_Krp(Iso[i][1])

            intg_c = imf_c*phi_fw1(Iso[i][2])*phi_fw2(Iso[i][3])*phi_fw3(Iso[i][4])
            p_c = trapz(intg_c,Iso[i][1])

            wr.append(str(p_c))

        return ([j,wr])

    def ai_samp(self, ID, Name):


        ### Data for STAN ###
        dats = {'Nj' : self.N_dat,
                'Ni' : self.NIso,
                'Pij': self.P_ij,
                'Cij': self.C_ij  }

        ############ Running pystan ############

        sm = stan.build(code, data=dats, random_seed=1234)
        fit = sm.sample(num_samples=self.N_smp, num_chains=self.N_wlk, num_warmup=200)
        self.fit = fit
        a_sp = fit["a"].T

        ######### Saving the MCMC sample #########

        N_iso = len(a_sp[0])

        a_perc = array([ percentile(ai,[10,50,90]) for ai in a_sp.T])       ##  10th, 50th, 90th percentiles

        sfh=array([self.Z_age_isos[:,0], self.Z_age_isos[:,1], a_perc[:,0], a_perc[:,1], a_perc[:,2] ]).T

        ##
        hd='Z,Log_age,p10,p50,p90'
        filename = ID+"_ai"+Name+"_Niter"+str(len(a_sp))+".txt"
        savetxt(self.out_dir + filename, sfh, header=hd, fmt="%.6f", delimiter=",",comments='')
        
        return filename


In [ ]:
class SFH(Base):
    def __init__(self,df=None,N_wlk=20, N_smp=500, fw1_lim=30.,fw2_lim=30., fw3_lim=30.0, 
                 A_fw1=0, A_fw2=0, A_fw3=0, sig_fw1=0.1,sig_fw2=0.1, sig_fw3=0.1, 
                 dismod=29.67,isofiles='', isodir=None, ph_sup=100,m_inf=0.1,
                 IMF='Krp',parallel=True, out_dir='.'):
        
        self.out_dir = out_dir
        
        self.N_wlk, self.N_smp = N_wlk, N_smp
        
        self.A_fw1, self.A_fw2, self.A_fw3 = A_fw1, A_fw2, A_fw3
        
        self.fw_lims = array([fw1_lim,fw2_lim,fw3_lim])
        
        self.sig_fw = array([sig_fw1,sig_fw2,sig_fw3])
        
        if isodir is None:
            isodir = f'{data_dir}/test_files/Isochrone.test'
        
        ########### Reading Isochrones ###########
        if (isofiles != ''):
            filelist = []
            f = open(isofiles, "r")
            lines = f.readlines()
            for line in lines:
                filelist.append(line.replace('\n',''))
        
            self.filelist = filelist
            f.close()
        else:
            filelist = glob.glob(os.path.join(isodir, "*.isoc"))
            self.filelist = filelist
        
        #               F435W  F555W  F814W
        #   ph   mass   mag1   mag2   mag3  Z  log_age
        iso = array([ loadtxt(k) for k in sorted(filelist) ], dtype=object)
        
        self.ages = [float(i.split('Myr')[0].split('AGE')[1]) for i in self.filelist]
        
        self.Z_age_isos = array([ loadtxt(k)[0][5:7] for k in sorted(self.filelist) ], dtype=object)
        
        ph_sup = 100
        m_inf = 0.1
        
        for l in range(len(iso)):
            iso[l] = iso[l][where( (iso[l].T[1]>=m_inf) & (iso[l].T[0]<=ph_sup))]       ## mass Truncation & stellar phase Truc
            iso[l] = iso[l].T
        
        self.Iso = iso
        ################################### DATA #######################################
        
        #  0      1       2        3         4          5         6          7
        #  RA    DEC     fw1   fw1_error    fw2     fw2_error    fw3     fw3_error
        if df is None:
            df = pd.read_fwf(f"{data_dir}/test_files/data.test", sep=' ')
            col_dict = {'RA_HST'  : 'RA',
                        'DEC_HST' : 'DEC',
                        'F435W'   : 'fw1',
                        'eF435W'  : 'fw1_error',
                        'F555W'   : 'fw2',
                        'eF555W'  : 'fw2_error',
                        'F814W'   : 'fw3',
                        'eF814W'  : 'fw3_error'}

            df = df.rename(columns=col_dict)
        keys = ['RA','DEC','fw1','fw1_error','fw2','fw2_error','fw3','fw3_error']
        try:
            dt = df[keys]
        except:
            print("Data Frame input keys: ", keys)
            raise Exception("Input data keys don't match!")
        
        step = int(1)
        dat = dt.values.astype(float)
        msg = "from %d... (FW1 <= %.2lf) (FW2 <= %.2lf) (FW3 <= %.2lf)" % (len(dat), fw1_lim, fw2_lim, fw3_lim)
        
        # Completeness Filtering
        dat = dat[where((dat[:,2] < fw1_lim) & (dat[:,4]  < fw2_lim) & (dat[:,6]  < fw3_lim) )]        # Truncate by apparent magnitude
        
        #dat = dat[where((dat[:,4]  < fw2_lim))]  
        w_dat = dat[:,0]
        
        # Adding Extinction
        dat[:,2] -= A_fw1+dismod
        dat[:,4] -= A_fw2+dismod
        dat[:,6] -= A_fw3+dismod
        
        print ("Selecting %d %s" % (len(dat), msg))
        dat = dat[::step]
        dat = dat
        self.dat = dat.T
        
        self.N_dat = len(self.dat[0])
        self.NIso = len(self.Iso)
        
        self.parallel = parallel
        self.IMF = IMF

    def __call__(self):
        ########################### Execution Routines #################################
        
        start = time.time()
        print ("Starting Pij, Cij computation")
        if (self.parallel):
            print ("\tParallel mode...")
        
            ID = str(int(time.time()))
        
            self.Pij_reslt = self.P_ij_map(ID)
            self.P_ij, Pij_name = self.Pij_reslt[0], self.Pij_reslt[1]
            
            Name = Pij_name[Pij_name.find("_Pij")+4:Pij_name.find(".txt")]
        
            self.C_ij = self.C_ij_map(ID)
        
        else:
            print ("\tSequential mode... Not available for the moment")
           # P_ij(dat, N_dat, r_int, iso, N_iso)
        print ("Finished                                ")
        
        
        end = time.time()
        
        elapsed = end - start
        
        print ("Elapsed time: %02d:%02d:%02d" % (int(elapsed / 3600.), int((elapsed % 3600)/ 60.), elapsed % 60))
        
        ###########################################
        filename = self.ai_samp(ID, Name)
        print("Completed!!!")
        return filename


In [ ]:
plt.rcParams['image.cmap'] = 'jet'
plt.rcParams['image.origin'] = 'lower'
plt.rcParams['figure.figsize'] = (7,5)
plt.rcParams['axes.titlesize'] = plt.rcParams['axes.labelsize'] = 35
plt.rcParams['xtick.labelsize'] = plt.rcParams['ytick.labelsize'] = 35

font1 = {'family': 'sans-serif', 'color': 'black', 'weight': 'normal', 'size': '15'}
font2 = {'family': 'sans-serif', 'color': 'black', 'weight': 'normal', 'size': '25'}

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})


In [ ]:
regions_dict = {}
with open('regions90_ngc628.json') as json_file:
    data = json.load(json_file)

regions_dict.update(data)

with open('../data/DS9 regions/bubbles.reg') as f:
    dat = f.readlines()

bubbles_dict = {}
for n,i in enumerate(dat[3:]):
    ra = float(i.split(',')[0][7:])
    dec = float(i.split(',')[1])
    radius = float(i.split(',')[2].split('"')[0])
    bubbles_dict[f'bubble_{n}'] = {'ra' : ra,
                              'dec': dec,
                              'radius': radius}
regions_dict.update(bubbles_dict) ;
regions_dict.update({ 'ngc628' : {'ra'   : 24.1738983, 
                       'dec'  : 15.7836543,
                       'F115W': 25.42, 
                       'F150W': 24.55,
                       'F200W': 23.56}})
Av_dict = { 
            'f275w': 2.02499,
            'f336w': 1.67536,
            'f435w': 1.33879,
            'f555w': 1.03065,
            'f814w': 0.59696,
    
            'f090w': 0.583,
            'f115w': 0.419,
            'f150w': 0.287,
            'f200w': 0.195,
    
            'f438w': 1.34148,
            'f606w': 0.90941,
            'f814w': 0.59845
          };

In [ ]:
def jwst_data(tab, ra_center=0, dec_center=0,r_out=24/3600, ang=245.00492,region_type='box'):

    if region_type == 'box':
        tab_filt = box(tab,'ra','dec',ra_center, dec_center, 0,0,r_out/3600, r_out/3600, ang)
    elif region_type == 'circle':
        tab['r'] = angular_separation(tab['ra']*u.deg,tab['dec']*u.deg,
                                      ra_center*u.deg, dec_center*u.deg).to(u.arcsec).value
        tab_filt = tab[tab['r']<=r_out]
    df = tab_filt.to_pandas()
    col_dict = {'ra'                 : 'RA',
                'dec'                : 'DEC',
                'mag_vega_F115W'     : 'fw1',
                'mag_err_F115W'      : 'fw1_error',
                'mag_vega_F150W'     : 'fw2',
                'mag_err_F150W'      : 'fw2_error',
                'mag_vega_F200W'     : 'fw3',
                'mag_err_F200W'      : 'fw3_error'}
    
    df_filt = df.rename(columns = col_dict)
    
    return df_filt

In [ ]:
def hst_data(tab, ra_center=0, dec_center=0,r_out=24/3600, ang=245.00492,region_type='circle'):

    comp_f115w = models.Polynomial1D(2,c0=27.32470673, c1=1.21351676, c2=-43.98416336)
    comp_f150w = models.Polynomial1D(2,c0=24.30894105, c1=17.79650983, c2=-57.09052021)
    comp_f200w = models.Polynomial1D(2,c0=25.84455633, c1=6.86355348, c2=-90.87952389)

    if region_type == 'box':
        tab_filt = box(tab,'ra','dec',ra_center, dec_center,0,0, r_out/3600, r_out/3600, ang)
        
    elif region_type == 'circle':
        tab['r'] = angular_separation(tab['ra']*u.deg,tab['dec']*u.deg,
                                      ra_center*u.deg, dec_center*u.deg).to(u.arcsec).value
        tab_filt = tab[tab['r']<=r_out]
    else:
        print(f'{region_type} NOT available!')
        
    df = tab_filt.to_pandas()
    col_dict = {'ra_1'                 : 'RA',
                'dec_1'                : 'DEC',
                'mag_vega_F435W'       : 'fw1',
                'mag_err_F435W'        : 'fw1_error',
                'mag_vega_F555W'       : 'fw2',
                'mag_err_F555W'        : 'fw2_error',
                'mag_vega_F814W'       : 'fw3',
                'mag_err_F814W'        : 'fw3_error'}
    
    df_filt = df.rename(columns = col_dict)
    fw1_lim = 30#comp_f115w(np.nanmedian(df['crowd_1']))
    fw2_lim = 27#comp_f150w(np.nanmedian(df['crowd_2']))
    fw3_lim = 30#comp_f200w(np.nanmedian(df['crowd']))
    
    return df_filt, [fw1_lim, fw2_lim, fw3_lim], [ra_center, dec_center]

In [ ]:
x = tab['f5007']
y = 10**((tab['m5007'].value.data + 13.74)/-2.5)

plt.scatter(x,y/x)

In [ ]:
x = tab['m5007'] - 28.1
y = tab['f5007'].value.data/tab['fHa'].value.data

fig, ax = plt.subplots(figsize=(10,6))

ax.scatter(x,y, color='black')

x = np.linspace(-6, -0.5)
y = -0.37*x - 1.16

ax.plot(x,10**y,'--r')

ax1 = ax.twiny()
ax1.plot(x + 28.1,10**y, '--r')


ax.set_yscale('log')

# Ensure the ticks are displayed as integers

ax.yaxis.set_major_formatter(ticker.ScalarFormatter())
#ax.yaxis.set_minor_formatter(ticker.NullFormatter())

ax.xaxis.set_minor_locator(AutoMinorLocator())

ax.yaxis.set_minor_locator(ticker.LogLocator(base=10.0, subs='auto', numticks=10))

ax.tick_params(which='both', length=15,direction="in", bottom=True, top=True,left=True, right=True, width=3)
ax.tick_params(which='minor', length=8, width=1)

ax.set_ylim(0.1,120)
ax.set_xlim(-5.2,-0.5)
ax1.set_xlim(-5.2 + 28.1,-0.5 + 28.1)


ax.set_xlabel(r'$M_{[OIII]}$')
ax1.set_xlabel(r'$m_{[OIII]}$', labelpad = 15)
ax.set_ylabel(r'$I_{[OIII]}/I_{H_{\alpha} + [NII]}$')         

plt.show()

In [ ]:
pred_SII = 10**(-0.4)*tab['fHa'].value

In [ ]:
pred_SII.max()

In [ ]:
fig, ax = plt.subplots(figsize = (10,6))

ax.hist(pred_SII, color='blue')

ax.set_xlabel(r'$I_{[SII]\lambda 6717 + [SII]\lambda 6731}$')
ax.set_ylabel('Count')

# **Completeness**

In [ ]:
@models.custom_model
def pritchet(m, alpha=0.5, m_50=22):
    return 0.5*(1 - alpha*(m - m_50)/np.sqrt(1 + alpha**2*(m-m_50)**2))

In [ ]:
fs = glob.glob(f'../photometry/ngc628/completeness/*.fake')
output_dir = '../photometry/ngc628/completeness'

In [ ]:
hdul = fits.open('../photometry/ngc628/completeness/F200W_i2d.fits')
wcs = WCS(hdul[1].header)

In [ ]:
for f in fs:
    cmd = f"python {script_dir}/to_table_fake.py --f {output_dir}/{f.split('/')[-1]}"
    cmd += f" --c {output_dir}/out.columns"
    cmd += f" --o {f.split('/')[-1][:-5]}"
    out = subprocess.run([cmd], shell=True)
    
    phot_table = Table.read(f"{output_dir}/{f.split('/')[-1][:-5]}.fits")

    positions = np.transpose([phot_table['x'] - 0.5, phot_table['y']-0.5])

    coords = np.array(wcs.pixel_to_world_values(positions))

    phot_table['ra']  = coords[:,0]
    phot_table['dec'] = coords[:,1]

    # Filtering stellar photometry catalog using Warfield et.al (2023)
    phot_table1 = phot_table[ (phot_table['obj_sharpness']**2<= 0.01) &
                            (phot_table['obj_crowd']<= 0.5) &
                            (phot_table['type'] <= 2)]
    flag_keys = []
    for key in phot_table1.keys():
        if 'flag' in key:
            flag_keys.append(key)
    for i in flag_keys:
        phot_table1  = phot_table1[phot_table1[i]<=2]

    SNR_keys = []
    for key in phot_table1.keys():
        if 'SNR' in key:
            SNR_keys.append(key)
    for i in SNR_keys:
        phot_table1  = phot_table1[phot_table1[i]>=5]
        
    phot_table.write(f"{output_dir}/{f.split('/')[-1][:-5]}.fits", overwrite=True)
    phot_table1.write(f"{output_dir}/{f.split('/')[-1][:-5]}_filt.fits", overwrite=True)

In [ ]:
ms_f200ws = []
mag_tol = 0.1
for i in np.round(np.arange(20,30.01,0.5),1):
    f_in = glob.glob(f'../photometry/ngc628/completeness/fake_fake_20_20_{i}_f115w_f150w_f200w_??.fits')
    f_out = glob.glob(f'../photometry/ngc628/completeness/fake_fake_20_20_{i}_f115w_f150w_f200w_??_filt.fits')
    t = []
    for f in f_in:
        tab = Table.read(f)
        t.append(tab)
    tab_in = vstack(t)
    
    t = []
    for f in f_out:
        tab = Table.read(f)
        t.append(tab)
    tab_out = vstack(t)  
    ms_f200w = []
    for key, data in bubbles.items():
        ra = data['ra']
        dec = data['dec']
        tab_in['r'] = angular_separation(ra*u.deg,dec*u.deg,tab_in['ra']*u.deg, tab_in['dec']*u.deg).to(u.arcsec).value
        c_in = len(tab_in[tab_in['r']<=data['radius']])
        if c_in>0:
            tab_out =  tab_out[(tab_out['mag_vega_F200W']>i-mag_tol) & (tab_out['mag_vega_F200W']<i+mag_tol)]

            tab_out['r'] = angular_separation(ra*u.deg,dec*u.deg,tab_out['ra']*u.deg, tab_out['dec']*u.deg).to(u.arcsec).value
            c_out = len(tab_out[tab_out['r']<=data['radius']])
            ms_f200w.append(c_out/c_in)    
        else:
            ms_f200w.append(np.nan)   
    ms_f200ws.append(ms_f200w)
   

In [ ]:
ms_f200ws = np.array(ms_f200ws) 

In [ ]:
ms_f150ws = []
for i in np.round(np.arange(20,30.01,0.5),1):
    f_in = glob.glob(f'../photometry/ngc628/completeness/fake_fake_20_{i}_20_f115w_f150w_f200w_??.fits')
    f_out = glob.glob(f'../photometry/ngc628/completeness/fake_fake_20_{i}_20_f115w_f150w_f200w_??_filt.fits')
    
    if len(f_in)<1 or len(f_out)<1:
        break
    t = []
    for f in f_in:
        tab = Table.read(f)
        t.append(tab)
    tab_in = vstack(t)
    
    t = []
    for f in f_out:
        tab = Table.read(f)
        t.append(tab)
    tab_out = vstack(t)  
    ms_f150w = []
    for j in range(90):
        ra = regions_dict[f'reg_{j}']['ra']
        dec = regions_dict[f'reg_{j}']['dec']
        
        c_in = len(box(tab_in,'ra','dec',ra,dec,0,0,24/3600,24/3600))
        tab_out =  tab_out[(tab_out['mag_vega_F150W']>i-mag_tol) & (tab_out['mag_vega_F150W']<i+mag_tol)]
        c_out = len(box(tab_out,'ra','dec',ra,dec,0,0,24/3600,24/3600))
        print(c_out,c_in)
        ms_f150w.append(c_out/c_in)
        
    ms_f150ws.append(ms_f150w)
    

In [ ]:
ms_f150ws = np.array(ms_f150ws)

In [ ]:
f_in

In [ ]:
ms_f115ws = []
for i in np.round(np.arange(20,30.01,0.5),1):
    f_in = glob.glob(f'../photometry/ngc628/completeness/fake_fake_{i}_20_20_f115w_f150w_f200w_??.fits')
    f_out = glob.glob(f'../photometry/ngc628/completeness/fake_fake_{i}_20_20_f115w_f150w_f200w_??_filt.fits')
    
    if len(f_in)<1 or len(f_out)<1:
        break
    t = []
    for f in f_in:
        tab = Table.read(f)
        t.append(tab)
    tab_in = vstack(t)
    
    t = []
    for f in f_out:
        tab = Table.read(f)
        t.append(tab)
    tab_out = vstack(t)  
    ms_f115w = []
    for j in range(90):
        ra = regions_dict[f'reg_{j}']['ra']
        dec = regions_dict[f'reg_{j}']['dec']
        
        c_in = len(box(tab_in,'ra','dec',ra,dec,0,0,24/3600,24/3600))
        tab_out =  tab_out[(tab_out['mag_vega_F115W']>i-mag_tol) & (tab_out['mag_vega_F115W']<i+mag_tol)]
        c_out = len(box(tab_out,'ra','dec',ra,dec,0,0,24/3600,24/3600))
        ms_f115w.append(c_out/c_in)
        
    ms_f115ws.append(ms_f115w)

In [ ]:
ms_f115ws = np.array(ms_f115ws)

In [ ]:
ms_f115ws.shape

In [ ]:
f200w_m50 = []
f200w_a = []

init = pritchet()
fit = fitting.LevMarLSQFitter()
x = np.arange(20,30.01,0.5)
for dat in ms_f200ws.T:
    offset = dat.max()
    model = fit(init, x,dat/offset)
    if model.m_50.value>20 and model.m_50.value<30:
        f200w_m50.append(model.m_50.value)
        f200w_a.append(model.alpha.value)
    else:
        model = lambda x_: np.interp(x_,x,dat)
        f200w_m50.append(np.interp(0.5,dat[::-1],x[::-1]))
        f200w_a.append(np.nan)
        print("Fit Failed!!!")
    #plt.plot(x,dat)
    #plt.plot(x,model(x)*offset, '--r')
    #plt.show()
    
f200w_m50 = np.array(f200w_m50).reshape(15,6)
f200w_a = np.array(f200w_a).reshape(15,6)


f150w_m50 = []
f150w_a = []

init = pritchet()
fit = fitting.LevMarLSQFitter()
x = np.arange(20,30.01,0.5)
for dat in ms_f150ws.T:
    offset = dat.max()
    model = fit(init, x,dat/offset)
    if model.m_50.value>20 and model.m_50.value<30:
        f150w_m50.append(model.m_50.value)
        f150w_a.append(model.alpha.value)
    else:
        model = lambda x_: np.interp(x_,x,dat)
        f150w_m50.append(np.interp(0.5,dat[::-1],x[::-1]))
        f150w_a.append(np.nan)
        print("Fit Failed!!!")
    
f150w_m50 = np.array(f150w_m50).reshape(15,6)
f150w_a = np.array(f150w_a).reshape(15,6)


f115w_m50 = []
f115w_a = []

init = pritchet()
fit = fitting.LevMarLSQFitter()
x = np.arange(20,30.01,0.5)
for dat in ms_f115ws.T:
    offset = dat.max()
    model = fit(init, x,dat/offset)
    if model.m_50.value>20 and model.m_50.value<30:
        f115w_m50.append(model.m_50.value)
        f115w_a.append(model.alpha.value)
    else:
        model = lambda x_: np.interp(x_,x,dat)
        f115w_m50.append(np.interp(0.5,dat[::-1],x[::-1]))
        f115w_a.append(np.nan)
        print("Fit Failed!!!")

    
f115w_m50 = np.array(f115w_m50).reshape(15,6)
f115w_a = np.array(f115w_a).reshape(15,6)

In [ ]:
np.save("../SFH/ngc628/F115W_m50.npy",f115w_m50)
np.save("../SFH/ngc628/F150W_m50.npy",f150w_m50)
np.save("../SFH/ngc628/F200W_m50.npy",f200w_m50)

np.save("../SFH/ngc628/F115W_a.npy",f115w_a)
np.save("../SFH/ngc628/F150W_a.npy",f150w_a)
np.save("../SFH/ngc628/F200W_a.npy",f200w_a)

In [ ]:
f115w_m50 = np.load("../SFH/ngc628/F115W_m50.npy")
f150w_m50 = np.load("../SFH/ngc628/F150W_m50.npy")
f200w_m50 = np.load("../SFH/ngc628/F200W_m50.npy")

f115w_a = np.load("../SFH/ngc628/F115W_a.npy")
f150w_a = np.load("../SFH/ngc628/F150W_a.npy")
f200w_a = np.load("../SFH/ngc628/F200W_a.npy")

In [ ]:
x, y = np.mgrid[0:15,0:6]/1.0
x -= 7
y -= 2.5
x *= 24
y *= 24

r = np.sqrt(x**2 + y**2)

In [ ]:
r[:9,2]

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

init = models.Polynomial1D(2)
fit = fitting.LevMarLSQFitter()

ax.scatter(r.ravel(), f115w_m50.ravel(), color='blue')

model = fit(init, r.ravel(),f115w_m50.ravel())
ax.plot(r.ravel(), model(r.ravel()), color='blue')

ax.scatter(r.ravel(), f150w_m50.ravel(), color='green')

model = fit(init, r.ravel(),f150w_m50.ravel())
ax.plot(r.ravel(), model(r.ravel()), color='green')

ax.scatter(r.ravel(), f200w_m50.ravel(), color='red')

model = fit(init, r.ravel(),f200w_m50.ravel())
ax.plot(r.ravel(), model(r.ravel()), color='red')

ax.set_ylabel(r'$m_{50}$')
ax.set_xlabel(r'R (arcsec)')

ax.invert_yaxis()

ax.xaxis.set_major_locator(AutoLocator())
ax.xaxis.set_minor_locator(AutoMinorLocator())

ax.yaxis.set_major_locator(AutoLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())

ax.tick_params(which='both', length=15,direction="in", bottom=True, top=True,left=True, right=True, width=3)
ax.tick_params(which='minor', length=8, width=3)

In [ ]:
wcs        = WCS(fits.open("../data/JWST/F200W_i2d.fits")[1].header)
df_bubbles = pd.DataFrame.from_dict(bubbles_dict, orient='index')

In [ ]:
coords = []
for i in range(90):
    coords.append([regions_dict[f'reg_{i}']['ra'], regions_dict[f'reg_{i}']['dec']])
    
pos = np.array(wcs.world_to_pixel_values(coords))
origin = np.array(wcs.world_to_pixel_values([[24.1738983, 15.7836543]]))

bubble_coords = np.array([df_bubbles['ra'], df_bubbles['dec']]).T

bubble_pos = np.array(wcs.world_to_pixel_values(bubble_coords))

In [ ]:
bubble_pos[19] = np.array([9868.9608565 , 4261])

In [ ]:
f200w_m50_bubbles = griddata(pos, f200w_m50.ravel(), bubble_pos, method="cubic")
f150w_m50_bubbles = griddata(pos, f150w_m50.ravel(), bubble_pos, method="cubic")
f115w_m50_bubbles = griddata(pos, f115w_m50.ravel(), bubble_pos, method="cubic")

f200w_a_bubbles = griddata(pos, f200w_a.ravel(), bubble_pos, method="cubic")
f150w_a_bubbles = griddata(pos, f150w_a.ravel(), bubble_pos, method="cubic")
f115w_a_bubbles = griddata(pos, f115w_a.ravel(), bubble_pos, method="cubic")

In [ ]:
img = plt.imshow(f200w_m50.T, cmap='jet')
plt.colorbar(img)

In [ ]:
img = plt.imshow(f200w_a.T, cmap='jet')
plt.colorbar(img)

# **Isochrones**

## **HST**

In [ ]:
df_cmd_acs = pd.read_csv("../data/isochrones_master/cmd_hst_acs.csv")

In [ ]:
if not os.path.exists('../data/isochrones/HST_ACS'):
    os.mkdir('../data/isochrones/HST_ACS/')
else:
    os.system('rm ../data/isochrones/HST_ACS/*')
l = 1  
tab_cmd = []
for i, age in enumerate([ 6.,  6.1, 6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8 , 6.9, 7., 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7,
                          7.8, 7.9, 8.0]):
    
    if len(str(i+1))==1:
        i = '0' + str(i+1)
    else:
        i =  str(i+1)
    for met in [0.02]:
        
        temp = df_cmd_acs[df_cmd_acs['logAge']==age]
        temp = temp[temp['Zini']==met]
        
        # Interpolation using F115W-F200W vs F200W
        
        x = temp['F435Wmag'].values - temp['F814Wmag'].values 
        y = temp['F814Wmag'].values
        
        diff = np.array([0] + list(np.sqrt((x[1:]-x[:-1])**2 + (y[1:]-y[:-1])**2)))
        dist = []
        for i in range(len(diff)):
            dist.append(diff[:i+1].sum())
            
        temp['dist'] = dist
        temp = temp.sort_values('dist')
        
        x = temp['dist']
        x_new = np.arange(x.min(), x.max(),0.01)

        label = np.interp(x_new, x, temp['label'])
        Mini = np.interp(x_new, x, temp['Mini'])
        f435w = np.interp(x_new, x, temp['F435Wmag'])
        f555w = np.interp(x_new, x, temp['F555Wmag'])
        f814w = np.interp(x_new, x, temp['F814Wmag'])

        temp_interp = pd.DataFrame(zip(label, Mini, f435w, f555w, f814w), columns = ['label','Mini','F435Wmag', 'F555Wmag', 'F814Wmag'])
        temp_interp['Zini'] = temp['Zini'].max()
        temp_interp['logAge'] = temp['logAge'].max()
        temp = temp_interp
        
        temp = temp[['label', 'Mini', 'F435Wmag', 'F555Wmag', 'F814Wmag', 'Zini', 'logAge']]
        header = list(temp.keys())
        header[0] = '#' + header[0]
            
        if l<10:       
            temp.to_csv(f'../data/isochrones/HST_ACS/0{l}_PARSEC1.1_Z0.020_logAGE{age}Myr_HST_BVI.isoc',sep=' ',
                       index=None, header = header)
        else:
            temp.to_csv(f'../data/isochrones/HST_ACS/{l}_PARSEC1.1_Z0.020_logAGE{age}Myr_HST_BVI.isoc',sep=' ',
                       index=None, header = header)
        tab_cmd.append(temp)
        l+=1
tab_cmd = pd.concat(tab_cmd)

In [ ]:
tab_cmd['logAge'].max()

In [ ]:
temp = tab_cmd[np.round(tab_cmd['logAge'],1)==7.7]

In [ ]:
fig, ax = plt.subplots()

x = temp['F435Wmag'] - temp['F814Wmag']
y = temp['F814Wmag'] + 29.7416

ax.scatter(x,y,s=1)

ax.invert_yaxis()

## **JWST**

### **PARSEC+COLIBRI**

In [ ]:
with open("../data/isochrones_master/cmd_jwst_n.dat") as f:
    dat = f.readlines()

data = []

for i,d in enumerate(dat[13:]):
    if 'Zini' not in d and 'terminated' not in d:
        data.append([float(i) for i in d.split()])
        
df_cmd = pd.DataFrame(data,columns=dat[13][2:].split())
df_cmd = df_cmd.drop_duplicates(['Mini','logAge','label'])
df_cmd.to_csv("../data/isochrones_master/cmd_jwst.csv")

In [ ]:
df_cmd_jwst = pd.read_csv("../data/isochrones_master/cmd_jwst.csv")

In [ ]:
if not os.path.exists('../data/isochrones/JWST_0.003'):
    os.mkdir('../data/isochrones/JWST_0.003/')
else:
    os.system('rm ../data/isochrones/JWST_0.003/*')

interp = True
tabs = pd.DataFrame()
l = 1
for i, age in enumerate([6.8    ,  6.9    ,  7.     ,
                        7.1    ,  7.2    ,  7.3  ,  7.4    ,  7.5    ,  7.6    ,
                        7.7    ,  7.8    ,  7.9  ,  8.,   8.1    ,  8.2    ,
                        8.3    ,  8.4    ,  8.5, 8.6, 8.7, 8.8, 8.9, 9,
                        9.1, 9.2, 9.3,  9.4, 9.5, 9.6, 9.7, 9.8, 9.90001, 10.00001]):  

    if len(str(i+1))==1:
        i = '0' + str(i+1)
    else:
        i =  str(i+1)

    for met in [0.02, 0.003]:
        temp = df_cmd_jwst[df_cmd_jwst['logAge']==age]

        temp = temp[(temp['Zini']==met)]
        temp = temp.drop_duplicates(['Mini','logAge','label'])  
        label_frac = temp.groupby('label').count()['Mini'].values.astype(float)
        label_frac/= label_frac.sum()
        if interp:
            # Interpolation using F115W-F200W vs F200W
            temps = []
            for n,lb in enumerate(np.unique(temp['label'])):
                temp_lb = temp[temp['label']==lb]
                x = temp_lb['F115Wmag'].values - temp_lb['F200Wmag'].values 
                y = temp_lb['F200Wmag'].values

                diff = np.array([0] + list(np.sqrt((x[1:]-x[:-1])**2 + (y[1:]-y[:-1])**2)))
                dist = np.cumsum(diff)

                temp_lb['dist'] = dist
                temp_lb = temp_lb.sort_values('dist')

                x = temp_lb['dist']
                x_new = np.arange(x.min(), x.max(),0.01)

                label = np.interp(x_new, x, temp_lb['label'])
                Mini = np.interp(x_new, x, temp_lb['Mini'])
                f115w = np.interp(x_new, x, temp_lb['F115Wmag'])
                f150w = np.interp(x_new, x, temp_lb['F150Wmag'])
                f200w = np.interp(x_new, x, temp_lb['F200Wmag'])

                temp_interp = pd.DataFrame(zip(label, Mini, f115w, f150w, f200w), columns = ['label','Mini','F115Wmag', 'F150Wmag', 'F200Wmag'])
                temp_interp['Zini'] = temp['Zini'].max()
                temp_interp['logAge'] = temp['logAge'].max()
                temp_interp['label_frac']= label_frac[n]/len(temp_interp)
                temps.append(temp_interp)
            temp = pd.concat(temps)
        temp = temp[['label', 'Mini', 'F115Wmag', 'F150Wmag', 'F200Wmag', 'Zini', 'logAge','label_frac']]
        temp['label_frac'] /= temp['label_frac'].sum()
        header = list(temp.keys())
        header[0] = '#' + header[0]
            
        if (met==0.02 and age<=9) or (met==0.003 and age>9):
            if l<10:
                temp.to_csv(f'../data/isochrones/JWST_0.003/0{l}_PARSEC1.2S_Z0.02_logAGE{age}Myr_JWST_JHK.isoc',sep=' ',
                               index=None, header = header)
            else:
                temp.to_csv(f'../data/isochrones/JWST_0.003/{l}_PARSEC1.2S_Z0.02_logAGE{age}Myr_JWST_JHK.isoc',sep=' ',
                               index=None, header = header)
            l += 1
        if len(temp)>0:
            tabs = pd.concat([tabs, temp])
        

### **BaSTI**

In [ ]:
fs = glob.glob('../data/isochrones_master/basti_jairo/*/*/*')

In [ ]:
dfs = []
for f in fs:
    with open(f) as fi:
        dat = fi.readlines()
        
    Z_str = dat[4][dat[4].index('Z'):dat[4].index('Y')]
    Z = float(Z_str.split()[-1])
    age = np.log10(float(dat[4].split()[-1])*1e6)
    cols = dat[6].split()[1:]

    data = [[float(i) for i in d.split()] for d in dat[8:]]
    data = np.array(data)
    df = pd.DataFrame(data, columns=cols)
    df['Zini'] = Z
    df['logAge'] = age
    dfs.append(df)
df_cmd_jwst_basti = pd.concat(dfs)
df_cmd_jwst_basti.to_csv('../data/isochrones_master/cmd_jwst_basti_v2.csv', index=None)

In [ ]:
df_cmd_jwst_basti

In [ ]:
np.unique(df_cmd_jwst_basti['logAge'].values)

# **Star Formation History**

In [ ]:
import nest_asyncio
nest_asyncio.apply()
del nest_asyncio

# **Data**

## **HST-Simulated**

### **M = 1e4**

In [ ]:
files = glob.glob(f"../data/sim/HST_1e4/*.dat")
for n,file in enumerate(files):

    with open(file) as f:
        dat = f.readlines()

    data = []

    for i,d in enumerate(dat[13:]):
        if '#' not in d and 'terminated' not in d:
            data.append([float(i) for i in d.split()])
    if n==0:
        df = pd.DataFrame(data,columns=dat[13][2:].split())
    else:    
        df_ = pd.DataFrame(data,columns=dat[13][2:].split())
        df = pd.concat([df,df_])

In [ ]:
df_cmd = df.drop_duplicates(['Mini','age','label'])
df_cmd['logAge'] = np.round(np.log10(df_cmd['age']),1)
df_cmd.to_csv("../data/sim/HST_1e4/sim.csv", index=None)

### **M = 1e5**

In [ ]:
files = glob.glob(f"../data/sim/HST_1e5/*.dat")
for n,file in enumerate(files):

    with open(file) as f:
        dat = f.readlines()

    data = []

    for i,d in enumerate(dat[13:]):
        if '#' not in d and 'terminated' not in d:
            data.append([float(i) for i in d.split()])
    if n==0:
        df = pd.DataFrame(data,columns=dat[13][2:].split())
    else:    
        df_ = pd.DataFrame(data,columns=dat[13][2:].split())
        df = pd.concat([df,df_])

In [ ]:
df_cmd = df.drop_duplicates(['Mini','age','label'])
df_cmd['logAge'] = np.round(np.log10(df_cmd['age']),1)
df_cmd.to_csv("../data/sim/HST_1e5/sim.csv", index=None)

### **M = 1e6**

In [ ]:
files = glob.glob(f"../data/sim/HST_1e6/*.dat")
for n,file in enumerate(files):

    with open(file) as f:
        dat = f.readlines()

    data = []

    for i,d in enumerate(dat[13:]):
        if '#' not in d and 'terminated' not in d:
            data.append([float(i) for i in d.split()])
    if n==0:
        df = pd.DataFrame(data,columns=dat[13][2:].split())
    else:    
        df_ = pd.DataFrame(data,columns=dat[13][2:].split())
        df = pd.concat([df,df_])

In [ ]:
df_cmd = df.drop_duplicates(['Mini','age','label'])
df_cmd['logAge'] = np.round(np.log10(df_cmd['age']),1)
df_cmd.to_csv("../data/sim/HST_1e6/sim.csv", index=None)

### **Run**

In [ ]:
df_sim = pd.read_csv("../data/sim/HST_1e5/sim.csv")
df_sim = df_sim.drop_duplicates(['Mini','logAge','label'])

In [ ]:
df_filt = df_sim[(df_sim['logAge']==7.2) | (df_sim['logAge']==7.6) | (df_sim['logAge']==7.8) ].copy()
df_filt = df_filt.rename(columns = {'F435Wmag' : 'mag_vega_F435W',
                                    'F555Wmag' : 'mag_vega_F555W',
                                    'F814Wmag' : 'mag_vega_F814W',})

model_mag_err_f200w = models.Exponential1D(1.87077026e-10, 1.31284337e+00/0.88)
model_col_err = models.Exponential1D(9.72187779e-10, 1.39331541e+00/0.88)

df_filt['ra_1']      = 0
df_filt['dec_1']     = 0
df_filt['ra']        = 0
df_filt['dec']       = 0
df_filt['mag_err_1'] = 0.001
df_filt['mag_err_2'] = 0.001
df_filt['mag_err']   = 0.001

tab_sim = Table.from_pandas(df_filt)
Av = 0.
AF115  =  0.419*Av
AF150  =  0.287*Av
AF200  =  0.195*Av
dismod =  29.67

model_mag_err_f200w = models.Exponential1D(1.87077026e-10, 1.31284337e+00)
model_col_err = models.Exponential1D(9.72187779e-10, 1.39331541e+00)

tab_sim['mag_vega_F435W'] = tab_sim['mag_vega_F435W'] + dismod + AF115
tab_sim['mag_vega_F555W'] = tab_sim['mag_vega_F555W'] + dismod + AF150
tab_sim['mag_vega_F814W'] = tab_sim['mag_vega_F814W'] + dismod + AF200

tab_sim['crowd_1']        = 0.01
tab_sim['crowd_2']        = 0.01
tab_sim['crowd']          = 0.01

fw1_lim = 30
fw2_lim = 30
fw3_lim = 30

tab_sim_ = tab_sim.copy()
tab_sim = tab_sim[tab_sim['mag_vega_F435W']<fw1_lim]
tab_sim = tab_sim[tab_sim['mag_vega_F555W']<fw2_lim]
tab_sim = tab_sim[tab_sim['mag_vega_F814W']<fw3_lim]

In [ ]:
Mean_M

In [ ]:
np.log10(Mean_M*Ncorrs)

In [ ]:
c1 = C1[0]
c2 = C2[0]
M = np.linspace(Ml,Mc)  
N1 = np.trapz(c1*M**(-alpha1), M)

M = np.linspace(Mc,mu)  
N2 = np.trapz(c2*M**(-alpha2), M)

In [ ]:
ages.shape

In [ ]:
ind.shape

In [ ]:
ages = np.round(np.arange(7,8.1,0.1),1)
Ni = ages*0

Ni_ = df_sim.groupby('logAge').count()['age'].values
ages_ = np.unique(df_sim['logAge'])

ind = np.array([True if np.round(i,1) in ages_ else False for i in ages])
Ni[ind] = Ni_

ai = 1
Ni*=ai

alpha1 = 1.3
alpha2 = 2.3
Mc = 0.5
Ml = 0.1

Mx = np.array([df_sim[np.round(df_sim['logAge'],1)==i]['Mini'].max() for i in ages])

h2 = (Mx[ind][0]**(1.-alpha2)-Mc**(1.-alpha2))/(1.-alpha2)
h2 *= Mc**(alpha2-alpha1)
h1 = (Mc**(1.-alpha1)-Ml**(1.-alpha1))/(1.-alpha1)

C1 = Ni/(h1+h2)
C2 = C1*Mc**(alpha2-alpha1)

Ncorrs = np.array(Ni)

Mean_M = []

for c1, c2, mu in zip(C1,C2,Mx):
    M = np.linspace(Ml,Mc)  
    Mean_M1 = np.trapz(M*c1*M**(-alpha1), M)

    M = np.linspace(Mc,mu)  
    Mean_M2 = np.trapz(M*c2*M**(-alpha2), M)

    Mean_M.append(Mean_M1+Mean_M2)

Mean_M = np.array(Mean_M)

delta_t = 10**ages - 10**(ages-0.1)

SFRs = Mean_M/delta_t

fig, ax = plt.subplots(figsize=(10,7))

x_ = ages
y_ = SFRs

ax.bar(x_,y_, width=0.2)

ax.set_xlabel('log Age (yr)')
ax.set_ylabel('SFR')

ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='both', width=2,direction="in", top = True,right = True,
               bottom = True, left = True)
ax.tick_params(which='major', length=7,direction="in")
ax.tick_params(which='minor', length=4, color='black',direction="in")
#ax.set_yscale('log')

In [ ]:
np.log10(Mean_M)

## **JWST- Simulated**

### **M = 1e5**

In [ ]:
files = glob.glob(f"../data/sim/JWST_1e5/*.dat")

In [ ]:
for n,file in enumerate(files):

    with open(file) as f:
        dat = f.readlines()

    data = []

    for i,d in enumerate(dat[13:]):
        if '#' not in d and 'terminated' not in d:
            data.append([float(i) for i in d.split()])
    if n==0:
        df = pd.DataFrame(data,columns=dat[13][2:].split())
    else:    
        df_ = pd.DataFrame(data,columns=dat[13][2:].split())
        df = pd.concat([df,df_])

In [ ]:
df_cmd = df.drop_duplicates(['Mini','age','label','Z'])
df_cmd['logAge'] = np.round(np.log10(df_cmd['age']),1)
df_cmd.to_csv("../data/sim/JWST_1e5/sim.csv", index=None)

In [ ]:
df_sim = pd.read_csv("../data/sim/JWST_1e5/sim.csv")
df_sim = df_sim.drop_duplicates(['Mini','logAge','label','Z'])
#df_sim = df_sim[df_sim['logAge']>8.9]

In [ ]:
df_sim['logAge'].min(),df_sim['logAge'].max()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
model_mag_err_f200w = models.Exponential1D(1.87077026e-10, 1.31284337e+00/0.88)
model_col_err = models.Exponential1D(9.72187779e-10, 1.39331541e+00/0.88)

for age in np.arange(6.8,10.2,0.1):
    age = np.round(age,1)
    df_temp = df_sim[df_sim['logAge']==age]
    if len(df_temp)<1:
        print(f'Age = {age} has 0 stars')
      
    x = df_temp['F115Wmag'] - df_temp['F200Wmag']
    y = df_temp['F200Wmag'] + 29.7416
    ind = y<27
    
    x = x[ind]
    y = y[ind]
    
    x = np.random.normal(loc=x,scale=model_col_err(y))
    y = np.random.normal(loc=y,scale=model_mag_err_f200w(y))
    
    ax.scatter(x,y, s=0.2, label = f'Age = {age}')
  
y = np.arange(18,30.1,0.5)
x = y*0 -0.5
xerr = model_col_err(y)
yerr = model_mag_err_f200w(y)

ax.errorbar(x, y, yerr,xerr ,fmt='o', color = 'red', markersize=0.5, capsize=2) 

ax.set_ylim(17.5,31)
ax.set_xlim(-1, 2)
ax.set_ylabel('F200W')
ax.set_xlabel('F115W - F200W')
ax.invert_yaxis()
ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='both', length=15, direction="in", top = True,right = True, bottom = True, left = True, width=3)
ax.tick_params(which='minor', length=8, color='black',direction="in", width=3)

#plt.legend(ncols=3);

In [ ]:
ph_fracs = []
for age in np.arange(6.8,10.1,0.1):
    age = np.round(age,1)
    df_temp = df_sim[df_sim['logAge']==age]
      
    l_d = df_temp['label'].values
    x = df_temp['F115Wmag'].values - df_temp['F200Wmag'].values
    y = df_temp['F200Wmag'].values + 29.7416
    M = df_temp['Mini'].values
    
    ph_frac = []
    
    for n,i in enumerate(range(0,10,1)):               
        if n==3:
            
            ind = (y<28) & (l_d == int(n))    
            t = x[ind]
            
            ind = (y<28) & (l_d == int(n) &(M>8) )    
            t = x[ind]
            ph_frac.append(len(t))
            ind = (y<28) & (l_d == int(n) & (M>2.2) & (M<=8) )    
            t = x[ind]
            ph_frac.append(len(t))
            ind = (y<28) & (l_d == int(n) & (M<=2.2)  )    
            t = x[ind]
            ph_frac.append(len(t))
        else:
            ind = (y<28) & (l_d == int(n))    
            t = x[ind]
            ph_frac.append(len(t))

    ph_frac = np.array(ph_frac)

    ph_fracs.append(ph_frac)

ph_fracs = np.array(ph_fracs)

In [ ]:
labels = ['PMS','MS', 'SGB', 'RSG', 'RCHEB','RGB', 'CHEB1', 'CHEB2','CHEB3',
                 'EAGB', 'TPAGB', 'post-AGB']

In [ ]:
ph_frac_tot = ph_fracs.sum(axis=0)

ind = 2
plt.figure(figsize=(10,7))
l = np.arange(0,len(labels),1)

plt.plot(l[ind:],ph_frac_tot[ind:],'-o', label=f'{age}')

plt.xticks(l[ind:],labels[ind:], fontsize=12)
plt.ylabel('No of stars');

In [ ]:
10**5/(10**

### **M = 1e6**

In [ ]:
tab = Table.read('../photometry/ngc628/f115w_f150w_f200w_photometry.fits')
df_cmd_jwst = pd.read_csv("../data/isochrones_master/cmd_jwst.csv")
df = tab[(tab['mag_err_F115W']<0.2)  & (tab['mag_err_F150W']<0.2) & (tab['mag_err_F200W']<0.2)] 

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))

x = df['mag_vega_F115W']
y = df['mag_err_F115W']

init = models.Exponential1D()
fit = fitting.LevMarLSQFitter()
model_err_f115w = fit(init,x,y)

ax.scatter(x,y,s=0.01,color='blue')
ax.axvline([24.49524254],ls='--', color='blue')
ax.axvline([27.27155834],ls='-', color='blue')

x = df['mag_vega_F150W']
y = df['mag_err_F150W']

model_err_f150w = fit(init,x,y)
ax.scatter(x,y,s=0.01,color='green')
ax.axvline([23.5535029],ls='--', color='green')
ax.axvline([25.71734278],ls='-', color='green')

x = df['mag_vega_F200W']
y = df['mag_err_F200W']

model_err_f200w = fit(init,x,y)

x_ = np.linspace(x.min(), x.max())
y_ = model_err_f200w(x_)
ax.plot(x_,y_,'--k')

ax.scatter(x,y,s=0.001,color='red')
ax.axvline([22.7474432],ls='--', color='red')
ax.axvline([25.9781503],ls='-', color='red')


ax.xaxis.set_major_locator(AutoLocator())
ax.xaxis.set_minor_locator(AutoMinorLocator())

ax.yaxis.set_major_locator(AutoLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())

ax.tick_params(which='both', length=15,direction="in", bottom=True, top=True,left=True, right=True,width=3)
ax.tick_params(which='minor', length=8,width=3)

ax.set_xlabel('mag')
ax.set_ylabel('mag_err')

In [ ]:
col_err = np.sqrt(df['mag_err_F115W']**2 + df['mag_err_F200W']**2)
mag     = df['mag_vega_F200W']

init    = models.Exponential1D()
fit     = fitting.LevMarLSQFitter()
model_col_err = fit(init,mag,col_err)

In [ ]:
ph_fracs = []
for age in [10.0]:
    age = np.round(age,1)
    data = [] 
    
    with open(f"../data/sim/JWST_1e6/JWST_{age}.dat") as f:
        dat = f.readlines()

    for i,d in enumerate(dat[13:]):
        if '#' not in d and 'terminated' not in d:
            data.append([float(i) for i in d.split()])
    data = np.array(data)
    
    l_d = data[:,7]
    x = data[:,28] - data[:,30]
    y =  data[:,30] + 29.83
    
    ph_frac = []
    l = np.arange(0,10,1)
    
    for i in l:
        ind = (y<28) & (l_d == int(i))    
        t = x[ind]
        ph_frac.append(len(t))

    ph_frac = np.array(ph_frac)

    labels = ['PMS','MS', 'SGB', 'RGB', 'CHEB1', 'CHEB2','CHEB3',
                     'EAGB', 'TPAGB', 'post-AGB']
    ph_fracs.append(ph_frac)

ph_fracs = np.array(ph_fracs)

In [ ]:
len(y[y<y.min()+0.1])

In [ ]:
ph_fracs

In [ ]:
ph_frac_tot = ph_fracs.sum(axis=0)

ind = 0
l = np.arange(0,10,1)
labels = ['PMS','MS', 'SGB', 'RGB', 'CHEB1', 'CHEB2','CHEB3',
                     'EAGB', 'TPAGB', 'post-AGB']

plt.plot(l[ind:],ph_frac_tot[ind:],'-o', label=f'{age}')

plt.xticks(l[ind:],labels[ind:])
plt.ylabel('No of stars');

In [ ]:
sim_df_filt = sim_df[(sim_df['Log_age']>=9.0) &
                     (sim_df['Log_age']<=10.0)] 


In [ ]:
x = sim_df_filt['F115Wmag'] - sim_df_filt['F200Wmag']
y = sim_df_filt['F200Wmag'] + dismod
c = sim_df_filt['Log_age']
s = np.array([float(i) for i in sim_df_filt['label']])

fig, ax = plt.subplots(figsize=(10,10))

img = ax.scatter(x,y,c=c, cmap='jet',s=0.5)
ax.plot([0,2],[23.3345,23.3345],'--r')
ax.invert_yaxis()
ax.set_ylim(28,17)

plt.colorbar(img,ax=ax)
ax.set_xlim(0,2)

## **Simulated observation**

In [ ]:
@models.custom_model
def pritchet(m, alpha=0.5, m_50=22):
    return 0.5*(1 - alpha*(m - m_50)/np.sqrt(1 + alpha**2*(m-m_50)**2))

@models.custom_model
def pritchet_inv(p, alpha=0.5, m_50=22):
    return m_50 + (1/alpha)*(1-2*p)/np.sqrt(1-(1-2*p)**2)


In [ ]:
tab = Table.read('../photometry/ngc628/f115w_f150w_f200w_photometry.fits')
df_cmd_jwst = pd.read_csv("../data/isochrones_master/cmd_jwst.csv")
df = tab[(tab['mag_err_F115W']<0.2) & (tab['mag_err_F150W']<0.2) & (tab['mag_err_F200W']<0.2)] 

x = df['mag_vega_F115W']
y = df['mag_err_F115W']

init = models.Exponential1D()
fit = fitting.LevMarLSQFitter()
model_f115w = fit(init,x,y)

x = df['mag_vega_F150W']
y = df['mag_err_F150W']

model_f150w = fit(init,x,y)

x = df['mag_vega_F200W']
y = df['mag_err_F200W']

model_f200w = fit(init,x,y)

In [ ]:
df_sim = pd.read_csv("../data/sim/JWST_1e5/sim.csv")
df_sim = df_sim.drop_duplicates(['Mini','logAge','label','Z'])

In [ ]:
df_filt = df_sim[((df_sim['logAge']>=6.8) & (df_sim['logAge']<9) & (df_sim['Z']==0.02)) | 
                 ((df_sim['logAge']>=9)    & (df_sim['Z']==0.002)  & (df_sim['logAge']<10.1))].copy()

df_filt = df_filt.rename(columns = {'F115Wmag' : 'mag_vega_F115W',
                                    'F150Wmag' : 'mag_vega_F150W',
                                    'F200Wmag' : 'mag_vega_F200W'
                                   })

df_filt['ra']        = np.float64(0)
df_filt['dec']       = np.float64(0)

tab_sim = Table.from_pandas(df_filt)
Av = 0.19
AF115  =  0.419*Av
AF150  =  0.287*Av
AF200  =  0.195*Av
dismod =  29.83

tab_sim['mag_vega_F115W'] += dismod + AF115
tab_sim['mag_vega_F150W'] += dismod + AF150
tab_sim['mag_vega_F200W'] += dismod + AF200

tab_sim = tab_sim[tab_sim['mag_vega_F115W']<29]
tab_sim = tab_sim[tab_sim['mag_vega_F150W']<29]
tab_sim = tab_sim[tab_sim['mag_vega_F200W']<29]

tab_sfr = []
ages = np.unique(df_filt['logAge'])

mult = np.array([ 3,   3,   3,   3,   3,   3,   3,   3,   3,   3,   4,   4,   6,
                  9,  11,  14,  16,  20,  20,  20,  15,  20,  20,  20,  12,  24,
                 26,  27,  45,  55,  80, 105, 101])

print(len(ages), len(mult))
for i,age in enumerate(ages):
    tab_t = tab_sim[tab_sim['logAge']==age]
    for j in range(mult[i]):
        tab_sfr.append(tab_t)
tab_sfr = vstack(tab_sfr)

tab_sfr['mag_err_F115W'] = model_f115w(tab_sfr['mag_vega_F115W'])
tab_sfr['mag_vega_F115W'] = np.random.normal(loc=tab_sfr['mag_vega_F115W'], scale = tab_sfr['mag_err_F115W'])

tab_sfr['mag_err_F150W'] = model_f150w(tab_sfr['mag_vega_F150W'])
tab_sfr['mag_vega_F150W'] = np.random.normal(loc=tab_sfr['mag_vega_F150W'], scale = tab_sfr['mag_err_F150W'])

tab_sfr['mag_err_F200W'] = model_f200w(tab_sfr['mag_vega_F200W'])
tab_sfr['mag_vega_F200W'] = np.random.normal(loc=tab_sfr['mag_vega_F200W'], scale = tab_sfr['mag_err_F200W'])

In [ ]:
ages

In [ ]:
fig,axs = plt.subplots(1,2, figsize=(15,5))

x = tab_sim['mag_vega_F115W'] - tab_sim['mag_vega_F200W']
y = tab_sim['mag_vega_F200W']

ax = axs[0]
ax.scatter(x,y,s=1, color='black')
ax.invert_yaxis()

ax.set_xlabel('F115W-F200W')
ax.set_ylabel('F200W')

x = tab_sfr['mag_vega_F115W'] - tab_sfr['mag_vega_F200W']
y = tab_sfr['mag_vega_F200W']

ax.xaxis.set_major_locator(AutoLocator())
ax.xaxis.set_minor_locator(AutoMinorLocator())

ax.yaxis.set_major_locator(AutoLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())

ax.tick_params(which='both', length=15,direction="in", bottom=True, top=True,left=True, right=True,width=3)
ax.tick_params(which='minor', length=8,width=3)

ax = axs[1]
ax.scatter(x,y,s=0.01, color='black')

ax.invert_yaxis()

ax.set_xlabel(r'F115W-F200W')
ax.set_ylabel(r'F200W')


ax.xaxis.set_major_locator(AutoLocator())
ax.xaxis.set_minor_locator(AutoMinorLocator())

ax.yaxis.set_major_locator(AutoLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())

ax.tick_params(which='both', length=15,direction="in", bottom=True, top=True,left=True, right=True,width=3)
ax.tick_params(which='minor', length=8,width=3)

In [ ]:
bins = np.arange(0,1.01,0.01)
for j in range(10):
    tab_cuts = []
    for k in range(len(df_bubbles)):
        
        tab_reg = tab_sfr.copy()
        tab_reg['comp_frac'] = pritchet(f200w_a_bubbles.ravel()[k], f200w_m50_bubbles.ravel()[k])(tab_reg['mag_vega_F200W'])*  \
                               pritchet(f150w_a_bubbles.ravel()[k], f150w_m50_bubbles.ravel()[k])(tab_reg['mag_vega_F150W'])*  \
                               pritchet(f115w_a_bubbles.ravel()[k], f115w_m50_bubbles.ravel()[k])(tab_reg['mag_vega_F115W'])

        tab_reg['ra'] = regions_dict[f'bubble_{int(k)}']['ra']
        tab_reg['dec'] = regions_dict[f'bubble_{int(k)}']['dec']

        tabs = []
        for i in range(len(bins)-1):
            df_cut = tab_reg[(tab_reg['comp_frac']>bins[i]) & (tab_reg['comp_frac']<=bins[i+1])].to_pandas()
            df = df_cut.sample(int(np.round(len(df_cut)*bins[i+1],0)))

            tabs.append(Table.from_pandas(df))

        tab_cut = vstack(tabs) 
        tab_cuts.append(tab_cut)
    tab_cuts = vstack(tab_cuts)
    tab_cuts.write(f"../SFH/ngc628/sim/comp_sim_{j}.fits", overwrite=True)

In [ ]:
tab_test = df_sim[((df_sim['logAge']>=6.8) & (df_sim['logAge']<=9) & (df_sim['Z']==0.02)) 
                | ((df_sim['logAge']>9) & (df_sim['Z']==0.002) & (df_sim['logAge']<10.1))].copy()

Ni = tab_test.groupby('logAge').count()['age'].values
ages = np.unique(tab_test['logAge'])

ai = 1
Ni*=ai

alpha1 = 1.3
alpha2 = 2.3
Mc = 0.5
Ml = 0.1

Mx = np.array([tab_test[tab_test['logAge']==i]['Mini'].max() for i in ages])

h2 = (Mx**(1.-alpha2)-Mc**(1.-alpha2))/(1.-alpha2)
h2 *= Mc**(alpha2-alpha1)
h1 = (Mc**(1.-alpha1)-Ml**(1.-alpha1))/(1.-alpha1)

C1 = (Ni*0+1)/(h1+h2)
C2 = C1*Mc**(alpha2-alpha1)

Ncorrs_ = np.array(Ni)*mult

Mean_M = []

for c1, c2, mu in zip(C1,C2,Mx):
    M = np.linspace(Ml,Mc)  
    Mean_M1 = np.trapz(M*c1*M**(-alpha1), M)

    M = np.linspace(Mc,mu)  
    Mean_M2 = np.trapz(M*c2*M**(-alpha2), M)

    Mean_M.append(Mean_M1+Mean_M2)

Mean_M = np.array(Mean_M)

delta_t = 10**ages - 10**(ages-0.1)

SFRs = (Mean_M*Ncorrs_)/delta_t

fig, ax = plt.subplots(figsize=(10,7))

x_ = ages
y_ = SFRs

SFR_ref = SFRs
ax.step(x_,y_,where='mid')

ax.set_xlabel('log Age (yr)')
ax.set_ylabel('SFR')

ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='both', width=2,direction="in", top = True,right = True,
               bottom = True, left = True)
ax.tick_params(which='major', length=7,direction="in")
ax.tick_params(which='minor', length=4, color='black',direction="in")
ax.set_yscale('log')

In [ ]:
len(ages)

In [ ]:
df_cmd_jwst = pd.read_csv("../data/isochrones_master/cmd_jwst.csv")

In [ ]:
len(tab_cut)

In [ ]:
tab_test = tab_cut.to_pandas()
Ni = tab_test.groupby('logAge').count()['age'].values
ages = np.unique(tab_test['logAge'])

ai = 1
Ni*=ai

alpha1 = 1.3
alpha2 = 2.3
Mc = 0.5
Ml = 0.1

k = -1
fw1_lim = f115w_m50_bubbles.ravel()[k].ravel()[0]
fw2_lim = f150w_m50_bubbles.ravel()[k].ravel()[0]
fw3_lim = f200w_m50_bubbles.ravel()[k].ravel()[0]

df_cmd = df_cmd_jwst[((df_cmd_jwst['logAge']>=6.8) & (df_cmd_jwst['logAge']<9) & (df_cmd_jwst['Zini']==0.02)) | 
                    ((df_cmd_jwst['logAge']>=9) & (df_cmd_jwst['Zini']==0.002))].copy()

df_cmd['F115Wmag'] += dismod + AF115
df_cmd['F150Wmag'] += dismod + AF150
df_cmd['F200Wmag'] += dismod + AF200

df_cmd = df_cmd[(df_cmd['F115Wmag']<=fw1_lim) & 
                (df_cmd['F150Wmag']<=fw2_lim) & 
                (df_cmd['F200Wmag']<=fw3_lim)]

Mx = np.array([df_cmd[np.round(df_cmd['logAge'],1)==i]['Mini'].max() for i in ages])

Mlim = np.array([df_cmd[np.round(df_cmd['logAge'],1)==i]['Mini'].min() for i in ages])

C2 = (Ni*(1-alpha2))/(Mx**(1-alpha2) - Mlim**(1-alpha2))
C1 = C2*Mc**(alpha1 - alpha2)

Ncorrs = []
for c1, c2, mlim, ni in zip(C1,C2,Mlim, Ni):
    M = np.linspace(Ml,Mc)  
    Nlim1 = np.trapz(c1*M**(-alpha1), M)

    M = np.linspace(Mc,mlim)  
    Nlim2 = np.trapz(c2*M**(-alpha2), M)

    Ncorr = ni + Nlim1 + Nlim2
    Ncorrs.append(Ncorr)

Ncorrs = np.array(Ncorrs)

h2 = (Mx[0]**(1.-alpha2)-Mc**(1.-alpha2))/(1.-alpha2)
h2 *= Mc**(alpha2-alpha1)
h1 = (Mc**(1.-alpha1)-Ml**(1.-alpha1))/(1.-alpha1)

C1 = (Ni*0+1)/(h1+h2)
C2 = C1*Mc**(alpha2-alpha1)

Mean_M = []

for c1, c2, mu in zip(C1,C2,Mx):
    M = np.linspace(Ml,Mc)  
    Mean_M1 = np.trapz(M*c1*M**(-alpha1), M)

    M = np.linspace(Mc,mu)  
    Mean_M2 = np.trapz(M*c2*M**(-alpha2), M)

    Mean_M.append(Mean_M1+Mean_M2)

Mean_M = np.array(Mean_M)

delta_t = 10**ages - 10**(ages-0.1)

SFRs = (Mean_M*Ncorrs)/delta_t

x = ages
y = SFRs
fig, ax = plt.subplots(figsize=(10,7))

ax.step(x,y,where='mid', label='Completeness limited', color='red')
ax.step(x_,np.array(y_),where='mid', label = 'Simulated', color='black')

ax.set_xlabel(r'$\log (Age)$ [yr]')
ax.set_ylabel(r'SFR $[M_{\odot}.yr^{-1}]$')

ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='both', width=2,direction="in", top = True,right = True,
               bottom = True, left = True)
ax.tick_params(which='major', length=7,direction="in")
ax.tick_params(which='minor', length=4, color='black',direction="in")
ax.set_yscale('log')
ax.legend(fontsize=30)

In [ ]:
corr = Ncorrs_/Ncorrs

In [ ]:
 Ni

In [ ]:
df_cmd_jwst = pd.read_csv("../data/isochrones_master/cmd_jwst.csv")

In [ ]:
df_cmd = df_cmd_jwst.copy()
df_cmd = df_cmd_jwst[((df_cmd_jwst['logAge']>=6.8) & (df_cmd_jwst['logAge']<=9) & (df_cmd_jwst['Zini']==0.02)) | 
                    ((df_cmd_jwst['logAge']>9) & (df_cmd_jwst['Zini']==0.002))].copy()

df_cmd['F115Wmag'] += dismod + AF115
df_cmd['F150Wmag'] += dismod + AF150
df_cmd['F200Wmag'] += dismod + AF200

fig, ax = plt.subplots(figsize=(10,7))
Ncorrss = []
for tab_test in tab_cuts:

    fw1_lim, fw2_lim, fw3_lim = f115w_m50.ravel()[0], f150w_m50.ravel()[0], f200w_m50.ravel()[0]

    tab_test = tab_test[tab_test['mag_vega_F115W']<=fw1_lim]
    tab_test = tab_test[tab_test['mag_vega_F150W']<=fw2_lim]
    tab_test = tab_test[tab_test['mag_vega_F200W']<=fw3_lim]

    Ni = tab_test.to_pandas().groupby('logAge').count()['age'].values
    ages = np.unique(tab_test['logAge'])

    ai = 1
    Ni*=ai

    alpha1 = 1.3
    alpha2 = 2.3
    Mc = 0.5
    Ml = 0.1

    df_cmd = df_cmd_jwst[((df_cmd_jwst['logAge']>=6.8) & (df_cmd_jwst['logAge']<=9) & (df_cmd_jwst['Zini']==0.02)) | 
                    ((df_cmd_jwst['logAge']>9) & (df_cmd_jwst['Zini']==0.002))].copy()

    df_cmd['F115Wmag'] += dismod + AF115
    df_cmd['F150Wmag'] += dismod + AF150
    df_cmd['F200Wmag'] += dismod + AF200

    df_cmd = df_cmd[(df_cmd['F115Wmag']<=fw1_lim) & 
                    (df_cmd['F150Wmag']<=fw2_lim) & 
                    (df_cmd['F200Wmag']<=fw3_lim)]
    
    Mx = np.array([df_cmd[np.round(df_cmd['logAge'],1)==i]['Mini'].max() for i in ages])

    Mlim = np.array([df_cmd[np.round(df_cmd['logAge'],1)==i]['Mini'].min() for i in ages])

    C2 = (Ni*(1-alpha2))/(Mx**(1-alpha2) - Mlim**(1-alpha2))
    C1 = C2*Mc**(alpha1 - alpha2)

    Ncorrs = []
    for c1, c2, mlim, ni in zip(C1,C2,Mlim, Ni):
        M = np.linspace(Ml,Mc)  
        Nlim1 = np.trapz(c1*M**(-alpha1), M)

        M = np.linspace(Mc,mlim)  
        Nlim2 = np.trapz(c2*M**(-alpha2), M)

        Ncorr = ni + Nlim1 + Nlim2
        Ncorrs.append(Ncorr)

    Ncorrs = np.array(Ncorrs)
    
    Ncorrss.append(Ncorrs)

    h2 = (Mx[0]**(1.-alpha2)-Mc**(1.-alpha2))/(1.-alpha2)
    h2 *= Mc**(alpha2-alpha1)
    h1 = (Mc**(1.-alpha1)-Ml**(1.-alpha1))/(1.-alpha1)

    C1 = (Ni*0+1)/(h1+h2)
    C2 = C1*Mc**(alpha2-alpha1)

    Mean_M = []

    for c1, c2, mu in zip(C1,C2,Mx):
        M = np.linspace(Ml,Mc)  
        Mean_M1 = np.trapz(M*c1*M**(-alpha1), M)

        M = np.linspace(Mc,mu)  
        Mean_M2 = np.trapz(M*c2*M**(-alpha2), M)

        Mean_M.append(Mean_M1+Mean_M2)

    Mean_M = np.array(Mean_M)

    delta_t = 10**ages - 10**(ages-0.1)

    SFRs = (Mean_M*Ncorrs)/delta_t

    x = ages
    y = SFRs.value

    ax.step(x,y,where='mid')

    ax.set_xlabel('log Age (yr)')
    ax.set_ylabel('SFR')

    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.tick_params(which='both', width=2,direction="in", top = True,right = True,
                   bottom = True, left = True)
    ax.tick_params(which='major', length=7,direction="in")
    ax.tick_params(which='minor', length=4, color='black',direction="in")
    ax.set_yscale('log')

ax.step(x_,np.array(y_)*10,where='mid')
ax.legend(['Completeness limited','Total'])

In [ ]:
incomp_frac = Ncorrs_/np.median(Ncorrss,axis=0)

In [ ]:
f115w_frac = pritchet(f115w_a[0],f115w[0])(tab_cut['mag_vega_F115W']).value
f150w_frac = pritchet(f150w_a[0],f150w[0])(tab_cut['mag_vega_F150W']).value
f200w_frac = pritchet(f200w_a[0],f200w[0])(tab_cut['mag_vega_F200W']).value

fracs = f115w_frac*f150w_frac*f200w_frac
tab_corrs = []
for tab_cut in tab_cuts:
    tab_corr = []
    for f, row in zip(fracs,tab_cut):
        if f>0.01:
            mult = int(np.round(1/f,0))
            for i in range(mult):
                tab_corr.append(Table(row))

    tab_corr = vstack(tab_corr)
    
    tab_corrs.append(tab_corr)

In [ ]:
Ni.values

In [ ]:
df_cmd = df_cmd_jwst[((df_cmd_jwst['logAge']>=7) & (df_cmd_jwst['logAge']<=9) & (df_cmd_jwst['Zini']==0.02)) | 
                ((df_cmd_jwst['logAge']>9) & (df_cmd_jwst['Zini']==0.002))].copy()

df_cmd['F115Wmag'] += dismod + AF115
df_cmd['F150Wmag'] += dismod + AF150
df_cmd['F200Wmag'] += dismod + AF200

df_cmd = df_cmd[(df_cmd['F115Wmag']<=fw1_lim) & 
                (df_cmd['F150Wmag']<=fw2_lim) & 
                (df_cmd['F200Wmag']<=fw3_lim)]

fig, ax = plt.subplots(figsize=(10,7))

Ncorrss = []
for tab_test in tab_corrs:


    fw1_lim = tab_test['mag_vega_F115W'].max()
    fw2_lim = tab_test['mag_vega_F150W'].max()
    fw3_lim = tab_test['mag_vega_F200W'].max()

    tab_test = tab_test[tab_test['mag_vega_F115W']<=fw1_lim]
    tab_test = tab_test[tab_test['mag_vega_F150W']<=fw2_lim]
    tab_test = tab_test[tab_test['mag_vega_F200W']<=fw3_lim]

    print(fw1_lim, fw2_lim, fw3_lim)

    Ni = tab_test.to_pandas().groupby('logAge').count()['age'].values
    ages = np.unique(tab_test['logAge'])

    ai = 1
    Ni*=ai

    alpha1 = 1.3
    alpha2 = 2.3
    Mc = 0.5
    Ml = 0.1

    df_cmd = df_cmd[(df_cmd['F115Wmag']<=fw1_lim) & (df_cmd['F150Wmag']<=fw2_lim) & (df_cmd['F200Wmag']<=fw3_lim)]

    Mx = np.array([df_cmd[np.round(df_cmd['logAge'],1)==i]['Mini'].max() for i in ages])

    Mlim = np.array([df_cmd[np.round(df_cmd['logAge'],1)==i]['Mini'].min() for i in ages])

    C2 = (Ni*(1-alpha2))/(Mx**(1-alpha2) - Mlim**(1-alpha2))
    C1 = C2*Mc**(alpha1 - alpha2)

    Ncorrs = []
    for c1, c2, mlim, ni in zip(C1,C2,Mlim, Ni):
        M = np.linspace(Ml,Mc)  
        Nlim1 = np.trapz(c1*M**(-alpha1), M)

        M = np.linspace(Mc,mlim)  
        Nlim2 = np.trapz(c2*M**(-alpha2), M)

        Ncorr = ni + Nlim1 + Nlim2

        Ncorrs.append(Ncorr)
    Ncorrs = np.array(Ncorrs)
    
    Ncorrss.append(Ncorrs)

    h2 = (Mx[0]**(1.-alpha2)-Mc**(1.-alpha2))/(1.-alpha2)
    h2 *= Mc**(alpha2-alpha1)
    h1 = (Mc**(1.-alpha1)-Ml**(1.-alpha1))/(1.-alpha1)

    C1 = (Ni*0+1)/(h1+h2)
    C2 = C1*Mc**(alpha2-alpha1)

    Mean_M = []

    for c1, c2, mu in zip(C1,C2,Mx):
        M = np.linspace(Ml,Mc)  
        Mean_M1 = np.trapz(M*c1*M**(-alpha1), M)

        M = np.linspace(Mc,mu)  
        Mean_M2 = np.trapz(M*c2*M**(-alpha2), M)

        Mean_M.append(Mean_M1+Mean_M2)

    Mean_M = np.array(Mean_M)

    delta_t = 10**ages - 10**(ages-0.1)

    SFRs = (Mean_M*Ncorrs*incomp_frac)/delta_t

    x = ages
    y = SFRs.value

    ax.step(x,y,where='mid')
    ax.step(x_,y_,where='mid')

    ax.set_xlabel('log Age (yr)')
    ax.set_ylabel('SFR')

    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.tick_params(which='both', width=2,direction="in", top = True,right = True,
                   bottom = True, left = True)
    ax.tick_params(which='major', length=7,direction="in")
    ax.tick_params(which='minor', length=4, color='black',direction="in")
    ax.set_yscale('log')

ax.legend(['Incompleteness Corrected by adding stars','Total'])

In [ ]:
incomp_frac = Ncorrs_/np.median(Ncorrss,axis=0)
incomp_frac_std = np.std(Ncorrs_/Ncorrss,axis=0)

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))

ax.step(x_,y_,where='mid')

ax.errorbar(x=x_, y=y_, yerr=y_*incomp_frac_std,fmt='o', color = 'red', markersize=0.5, capsize=2 )
ax.set_xlabel('log Age (yr)')
ax.set_ylabel('SFR')

ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='both', width=2,direction="in", top = True,right = True,
               bottom = True, left = True)
ax.tick_params(which='major', length=7,direction="in")
ax.tick_params(which='minor', length=4, color='black',direction="in")
ax.set_yscale('log')

In [ ]:
incomp_frac_std

# **Run**

## **HST**

In [ ]:
df = pd.read_fwf('../BayeSFH-NGC0628/data/ngc628_bubble_ACS_F200_stars1_radec_p15_VEGA_galax.cat', sep= ' ').drop(columns = '#')
col_dict = {'RA_HST' : 'ra_1',
            'DEC_HST': 'dec_1',
            'F435W'  : 'mag_vega_F435W',
            'eF435W' : 'mag_err_1',
            'F555W'  : 'mag_vega_F555W',
            'eF555W' : 'mag_err_2',
            'F814W'  : 'mag_vega_F814W',
            'eF814W' : 'mag_err'}

df_test = df.rename(columns = col_dict)

In [ ]:
tab_test = Table.from_pandas(df_test)

In [ ]:
tab = Table.read('../photometry/ngc628/f435w_f555w_f814w.fits')

#tab = tab_test

Av     =  0.19
AF435  =  1.33879*Av
AF555  =  1.03065*Av
AF814  =  0.59696*Av
dismod =  29.7416
    
sig_i = 0.01
for reg in range(0,1):
    reg = -1
     # JWST
    r_in      = 0
    r_out     = 8.9 # 24
    ang       = 245.00492
    
    ra_center  = regions_dict['bubble_0']['ra']
    dec_center = regions_dict['bubble_0']['dec']
    
    df_filt, fws, cen_coords = hst_data(tab, ra_center, dec_center, r_out,
                                        region_type='circle')
    
    # HST
    
    sfh = SFH(df_filt, parallel=True, 
              isodir="../data/isochrones/HST_ACS/", 
              fw1_lim=fws[0],fw2_lim=fws[1], fw3_lim=fws[2],
              sig_fw1=sig_i, sig_fw2=sig_i, sig_fw3=sig_i,
              A_fw1=AF435, A_fw2=AF555, A_fw3=AF814,
             dismod=dismod)

    fname = sfh()
    df_sfh = pd.read_csv(fname)
    P_ij = np.array(sfh.P_ij)
    ai = df_sfh['p50'].values
    ages = df_sfh['Log_age'].values
    
    df_sfh.to_csv(f'../SFH/ngc628/HST/reg_{int(reg)}.csv', index = None)
    p_age = P_ij*ai
    P = []
    P_age = []
    for i in p_age:
        prob = i[i==i.max()]
        ind = np.where(i==i.max())[0][0]
        P.append(prob)
        P_age.append(ages[ind])
        
    df = pd.DataFrame(sfh.dat.T, columns=['RA','DEC','mag_vega_F435W',
                                    'mag_err_1', 'mag_vega_F555W',
                                    'mag_err_2','mag_vega_F814W',
                                    'mag_err'])
    
    df['Prob'] = P
    df['Log_Age'] = P_age
    df.to_csv(f'../SFH/ngc628/HST/reg_{int(reg)}_spatial.csv', index = None)
    
    #F435W-F555W CMD
    x = df['mag_vega_F435W'] - df['mag_vega_F555W'] + (AF435-AF555)
    y = df['mag_vega_F555W'] + dismod + AF555
    c = df['Log_Age']
    fig, ax = plt.subplots(figsize=(12, 10))

    img = ax.scatter(x,y,c=c,s=20, cmap='jet', label='_nolegend_')
    cb = plt.colorbar(img, ax=ax)
    ind = -1
    ax.scatter(sfh.Iso[ind][2]-sfh.Iso[ind][3] + (AF435-AF555), sfh.Iso[ind][3] + dismod+ AF555, 
                                               label = f'{ages[ind]}', color='black',s=5,alpha=0.8)
    ax.set_ylim(21, 28)
    ax.set_xlabel('F435W-F555W')
    ax.set_ylabel('F555W')
    ax.set_xlim(-1,2)
    ax.invert_yaxis()
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.tick_params(which='both', width=3,direction="in", top = True,right = True,
                   bottom = True, left = True)
    ax.tick_params(which='major', length=15,direction="in")
    ax.tick_params(which='minor', length=8, color='black',direction="in")
    
    fig.savefig(f'../SFH/ngc628/HST/plots/reg_{int(reg)}_F435W-F555W_CMD.png', bbox_inches='tight')
    plt.close(fig)
    
    #F555W-F814W CMD
    x = df['mag_vega_F555W'] - df['mag_vega_F814W'] + (AF555-AF814)
    y = df['mag_vega_F814W'] + dismod + AF814
    c = df['Log_Age']
    fig, ax = plt.subplots(figsize=(12, 10))

    img = ax.scatter(x,y,c=c,s=20, cmap='jet', label='_nolegend_')
    cb = plt.colorbar(img, ax=ax)
    ind = -1
    ax.scatter(sfh.Iso[ind][3]-sfh.Iso[ind][4] + (AF555-AF814), sfh.Iso[ind][4] + dismod+ AF814, 
                                               label = f'{ages[ind]}', color='black',s=5,alpha=0.8)
    ax.set_ylim(22, 28)
    ax.set_xlabel('F555W-F814W')
    ax.set_ylabel('F814W')
    ax.set_xlim(-1,3)
    ax.invert_yaxis()
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.tick_params(which='both', width=3,direction="in", top = True,right = True,
                   bottom = True, left = True)
    ax.tick_params(which='major', length=15,direction="in")
    ax.tick_params(which='minor', length=8, color='black',direction="in")
    
    fig.savefig(f'../SFH/ngc628/HST/plots/reg_{int(reg)}_F555W-F814W_CMD.png', bbox_inches='tight')
    plt.close(fig)
    
    #F435W-F814W CMD
    x = df['mag_vega_F435W'] - df['mag_vega_F814W'] + (AF435-AF814)
    y = df['mag_vega_F814W'] + dismod + AF814
    c = df['Log_Age']
    fig, ax = plt.subplots(figsize=(12, 10))

    img = ax.scatter(x,y,c=c,s=20, cmap='jet', label='_nolegend_')
    cb = plt.colorbar(img, ax=ax)
    ind = -1
    ax.scatter(sfh.Iso[ind][2]-sfh.Iso[ind][4] + (AF435-AF814), sfh.Iso[ind][4] + dismod+ AF814, 
                                               label = f'{ages[ind]}', color='black',s=5,alpha=0.8)
    ax.set_ylim(22, 28)
    ax.set_xlabel('F435W-F814W')
    ax.set_ylabel('F814W')
    ax.set_xlim(-1,4)
    ax.invert_yaxis()
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.tick_params(which='both', width=3,direction="in", top = True,right = True,
                   bottom = True, left = True)
    ax.tick_params(which='major', length=15,direction="in")
    ax.tick_params(which='minor', length=8, color='black',direction="in")
    
    fig.savefig(f'../SFH/ngc628/HST/plots/reg_{int(reg)}_F435W-F814W_CMD.png', bbox_inches='tight')
    plt.close(fig)
    
    df_out = pd.read_csv(fname)

    for u_p in range(2):
        fig, ax = plt.subplots(figsize=(12,8))

        N = len(df)
        Ni = N*ai/ai.sum()
        x = df_out['Log_age']
        y = Ni

        ax.step(x,y,where='mid', color='blue')

        ax.set_xlabel('log Age (yr)')
        ax.set_ylabel('No of stars')

        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        ax.tick_params(which='both', width=3,direction="in", top = True,right = True,
                       bottom = True, left = True)
        ax.tick_params(which='major', length=15,direction="in")
        ax.tick_params(which='minor', length=8, color='black',direction="in")
        if u_p==0:
            ax.set_yscale('log')
            fig.savefig(f'../SFH/ngc628/HST/plots/reg_{int(reg)}_counts_log.png', bbox_inches='tight')
        else:
            fig.savefig(f'../SFH/ngc628/HST/plots/reg_{int(reg)}_counts.png', bbox_inches='tight')
        plt.close(fig)
    
    # Probability to SFH
    N = len(df)
    Ni = N*ai/ai.sum()
    
    alpha1 = 1.3
    alpha2 = 2.3
    Mc = 0.5
    Ml = 0.1
    
    FW1 = fws[0] - AF435 - dismod
    FW2 = fws[1] - AF555 - dismod
    FW3 = fws[2] - AF814 - dismod
    
    Mx = np.array([i[1].max() for i in sfh.Iso])
    Mlim = np.array([i[1][(i[2]<=FW1) & (i[3]<=FW2) & (i[4]<=FW3)].min() for i in sfh.Iso])
    
    C2 = (Ni*(1-alpha2))/(Mx**(1-alpha2) - Mlim**(1-alpha2))
    C1 = C2*Mc**(alpha1 - alpha2)
    
    Ncorrs = []
    for c1, c2, mlim, ni in zip(C1,C2,Mlim, Ni):
        M = np.linspace(Ml,Mc)  
        Nlim1 = np.trapz(c1*M**(-alpha1), M)

        M = np.linspace(Mc,mlim)  
        Nlim2 = np.trapz(c2*M**(-alpha2), M)
    
        Ncorr = ni + Nlim1 + Nlim2
        
        Ncorrs.append(Ncorr)
    Ncorrs = np.array(Ncorrs)
    
    C2 = (Ni*(1-alpha2))/(Mx**(1-alpha2) - Mlim**(1-alpha2))
    C1 = C2*Mc**(alpha1 - alpha2)

    Mean_M = []

    for c1, c2, mu in zip(C1,C2,Mx):
        M = np.linspace(Ml,Mc)  
        Mean_M1 = np.trapz(M*c1*M**(-alpha1), M)

        M = np.linspace(Mc,mu)  
        Mean_M2 = np.trapz(M*c2*M**(-alpha2), M)
        
        Mean_M.append(Mean_M1+Mean_M2)
      
    Mean_M = np.array(Mean_M)
    
    delta_t = 10**ages- 10**(ages-0.1)
    
    SFRs = Mean_M/delta_t
    
    df_out['SFR'] = SFRs
    
    x = df_out['Log_age']
    y = SFRs*1e3
    fig, ax = plt.subplots(figsize=(12,8))
    ax.step(x,y,where='mid', color='blue')
    #ax.step(x_,y_,where='mid')

    ax.set_xlabel(r'$\log(Age)~[yr]$')
    ax.set_ylabel(r'$SFR~[[10^{-3}]M_{\odot}.yr^{-1}]$')

    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.tick_params(which='both', width=3,direction="in", top = True,right = True,
                   bottom = True, left = True)
    ax.tick_params(which='major', length=15,direction="in")
    ax.tick_params(which='minor', length=8, color='black',direction="in")
    ax.set_yscale('log')
    fig.savefig(f'../SFH/ngc628/HST/plots/reg_{int(reg)}_SFH_log.png', bbox_inches='tight')
    plt.close(fig)
    
    fig, ax = plt.subplots(figsize=(12,8))
    ax.step(x,y,where='mid', color='blue')
    #ax.step(x_,y_,where='mid')

    ax.set_xlabel(r'$\log(Age)~[yr]$')
    ax.set_ylabel(r'$SFR~[[10^{-3}]M_{\odot}.yr^{-1}]$')

    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.tick_params(which='both', width=3,direction="in", top = True,right = True,
                   bottom = True, left = True)
    ax.tick_params(which='major', length=15,direction="in")
    ax.tick_params(which='minor', length=8, color='black',direction="in")
    fig.savefig(f'../SFH/ngc628/HST/plots/reg_{int(reg)}_SFH.png', bbox_inches='tight')
    plt.close(fig)
    
    df_out.to_csv(f'../SFH/ngc628/HST/reg_{int(reg)}_SFH.csv', index = None)

In [ ]:
z = P_ij[3]
x = df_out['Log_age']
plt.step(x,z, where='mid')

## **JWST**

In [ ]:
import nest_asyncio
nest_asyncio.apply()
del nest_asyncio

In [ ]:
tab = Table.read("../photometry/ngc628/f115w_f150w_f200w_photometry.fits")
#tab = Table.read('../SFH/ngc628/sim/comp_sim_0.fits')

tab = tab[(tab['mag_err_F115W']<=0.2) & (tab['mag_err_F150W']<=0.2) & (tab['mag_err_F200W']<=0.2)]

Av     =  0.19
AF115  =  0.419*Av
AF150  =  0.287*Av
AF200  =  0.195*Av
dismod =  29.81
    
sig_i = 0.010
for i in range(50, 90):
    # JWST
    region = f'reg_{i}'
    
    data = regions_dict[region]
    
    r_in      = 0
    r_out     = 24 #data['radius']
    ang       = 245.00492
    
    ra_center  = data['ra']
    dec_center = data['dec']
    
    df_filt = jwst_data(tab, ra_center, dec_center, r_out, ang,  region_type='box')
    
    reg_id = int(region.split('_')[1])

    #fws = [f115w_m50_bubbles[reg_id], f150w_m50_bubbles[reg_id], f200w_m50_bubbles[reg_id]]
    fws = [f115w_m50.ravel()[reg_id], f150w_m50.ravel()[reg_id], f200w_m50.ravel()[reg_id]]
    #fws = [30,30,30]
    # JWST
    
    out_dir = '../SFH/ngc628/JWST_reg_obs/'
    os.makedirs(out_dir,exist_ok=True)
    os.makedirs(out_dir+'output/',exist_ok=True)
    os.makedirs(out_dir+'plots/',exist_ok=True)
    
    sfh = SFH(df_filt, parallel=True, isodir='../data/isochrones/JWST/', 
              fw1_lim=fws[0],fw2_lim=fws[1], fw3_lim=fws[2],
              sig_fw1=sig_i, sig_fw2=sig_i, sig_fw3=sig_i,
              A_fw1=AF115, A_fw2=AF150, A_fw3=AF200,
             dismod=dismod, out_dir=out_dir+'output/')

    fname = sfh()
    fname = out_dir +'output/'+ fname
    df_sfh = pd.read_csv(fname)
    P_ij = np.array(sfh.P_ij)
    
    ai_10 = df_sfh['p10'].values
    ai_10 = ai_10/ai_10.sum()
    
    ai = df_sfh['p50'].values
    ai = ai/ai.sum()
    
    ai_90 = df_sfh['p90'].values
    ai_90 = ai_90/ai_90.sum()
    
    ages = df_sfh['Log_age'].values
    df_sfh['Ni'] = ai*sfh.dat.T.shape[0]
    
    df_sfh.to_csv(f'{out_dir}/{region}.csv', index = None)
    p_age = P_ij*ai
    P = []
    P_age = []
    for i in p_age:
        prob = i[i==i.max()]/i.sum()
        ind = np.where(i==i.max())[0][0]
        P.append(prob)
        P_age.append(ages[ind])
        
    df = pd.DataFrame(sfh.dat.T, columns=['RA','DEC','mag_vega_F115W',
                                    'mag_err_1', 'mag_vega_F150W',
                                    'mag_err_2','mag_vega_F200W',
                                    'mag_err'])
    
    df['Prob'] = P
    df['Log_Age'] = P_age
    df.to_csv(f'{out_dir}/{region}_spatial.csv', index = None)
    
    #F115W-F150W CMD
    x = df['mag_vega_F115W'] - df['mag_vega_F150W'] + (AF115-AF150)
    y = df['mag_vega_F150W'] + dismod + AF150
    c = df['Log_Age']
    fig, ax = plt.subplots(figsize=(12, 10))

    img = ax.scatter(x,y,c=c,s=1, cmap='jet', label='_nolegend_')
    cb = plt.colorbar(img, ax=ax)
    ind = -1
    ax.scatter(sfh.Iso[ind][2]-sfh.Iso[ind][3] + (AF115-AF150), sfh.Iso[ind][3] + dismod+ AF150, 
                                               label = f'{ages[ind]}', color='black',s=5,alpha=0.8)
    ax.set_ylim(20, fws[1]+1)
    ax.set_xlim(-0.5,2)
    ax.set_xlabel('F115W-F150W')
    ax.set_ylabel('F150W')
    ax.invert_yaxis()
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.tick_params(which='both', width=3,direction="in", top = True,right = True,
                   bottom = True, left = True)
    ax.tick_params(which='major', length=15,direction="in")
    ax.tick_params(which='minor', length=8, color='black',direction="in")
    
    fig.savefig(f'{out_dir}/plots/{region}_F115W-F150W_CMD.png', bbox_inches='tight')
    plt.close(fig)
    
    #F150W-F200W CMD
    x = df['mag_vega_F150W'] - df['mag_vega_F200W'] + (AF150-AF200)
    y = df['mag_vega_F200W'] + dismod + AF200
    c = df['Log_Age']
    fig, ax = plt.subplots(figsize=(12, 10))

    img = ax.scatter(x,y,c=c,s=1, cmap='jet', label='_nolegend_')
    cb = plt.colorbar(img, ax=ax)
    ind = -1
    ax.scatter(sfh.Iso[ind][3]-sfh.Iso[ind][4] + (AF150-AF200), sfh.Iso[ind][4] + dismod+ AF200, 
                                               label = f'{ages[ind]}', color='black',s=5,alpha=0.8)
    ax.set_ylim(20, fws[2]+1)
    ax.set_xlim(-0.5,2)
    ax.set_xlabel('F150W-F200W')
    ax.set_ylabel('F200W')
    ax.invert_yaxis()
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.tick_params(which='both', width=3,direction="in", top = True,right = True,
                   bottom = True, left = True)
    ax.tick_params(which='major', length=15,direction="in")
    ax.tick_params(which='minor', length=8, color='black',direction="in")
    
    fig.savefig(f'{out_dir}/plots/{region}_F150W-F200W_CMD.png', bbox_inches='tight')
    plt.close(fig)
    
    #F115W-F200W CMD
    x = df['mag_vega_F115W'] - df['mag_vega_F200W'] + (AF115-AF200)
    y = df['mag_vega_F200W'] + dismod + AF200
    c = df['Log_Age']
    fig, ax = plt.subplots(figsize=(12, 10))

    img = ax.scatter(x,y,c=c,s=1, cmap='jet', label='_nolegend_')
    cb = plt.colorbar(img, ax=ax)
    ind = -1
    ax.scatter(sfh.Iso[ind][2]-sfh.Iso[ind][4] + (AF115-AF200), sfh.Iso[ind][4] + dismod+ AF200, 
                                               label = f'{ages[ind]}', color='black',s=5,alpha=0.8)
    ax.set_ylim(20, fws[2]+1)
    ax.set_xlabel('F115W-F200W')
    ax.set_ylabel('F200W')
    ax.set_xlim(-0.5,3)
    ax.invert_yaxis()
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.tick_params(which='both', width=3,direction="in", top = True,right = True,
                   bottom = True, left = True)
    ax.tick_params(which='major', length=15,direction="in")
    ax.tick_params(which='minor', length=8, color='black',direction="in")
    
    fig.savefig(f'{out_dir}/plots/{region}_F115W-F200W_CMD.png', bbox_inches='tight')
    plt.close(fig)
    
    
    df_out = pd.read_csv(fname)

    fig, ax = plt.subplots(figsize=(12,8))

    N = len(df)
    Ni = N*ai
    x = df_out['Log_age']
    y = Ni

    ax.step(x,y,where='mid', color='blue')

    ax.set_xlabel('log Age (yr)')
    ax.set_ylabel('No of stars')

    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.tick_params(which='both', width=3,direction="in", top = True,right = True,
                   bottom = True, left = True)
    ax.tick_params(which='major', length=15,direction="in")
    ax.tick_params(which='minor', length=8, color='black',direction="in")
    ax.set_yscale('log')
    fig.savefig(f'{out_dir}/plots/{region}_counts.png', bbox_inches='tight')
    plt.close(fig)
    
    # Probability to SFH
    N = len(df)
    Ni = N*ai
    
    alpha1 = 1.3
    alpha2 = 2.3
    Mc = 0.5
    Ml = 0.1
    
    FW1 = fws[0] - AF115 - dismod
    FW2 = fws[1] - AF150 - dismod
    FW3 = fws[2] - AF200 - dismod
    
    Mx = np.array([i[1].max() for i in sfh.Iso])
    Mlim = []
    for i in sfh.Iso:
        j = i[1][(i[2]<=FW1) & (i[3]<=FW2) & (i[4]<=FW3)]
        if len(j)>0:
            Mlim.append(j.min())
        else:
            Mlim.append(0)
    Mlim = np.array(Mlim)
    
    C2 = (Ni*(1-alpha2))/(Mx**(1-alpha2) - Mlim**(1-alpha2))
    C1 = C2*Mc**(alpha1 - alpha2)
    
    # Ncorrs
    Ncorrs = []
    for c1, c2, mlim, ni in zip(C1,C2,Mlim, Ni):
        if mlim!=0:
            M = np.linspace(Ml,Mc)  
            Nlim1 = np.trapz(c1*M**(-alpha1), M)

            M = np.linspace(Mc,mlim)  
            Nlim2 = np.trapz(c2*M**(-alpha2), M)

            Ncorr = ni + Nlim1 + Nlim2
        else:
            Ncorr = 0
        Ncorrs.append(Ncorr)
    Ncorrs = np.array(Ncorrs)
    
    # N errors
    Ni_up = (ai_90-ai)*Ni
    Ni_down = (ai-ai_10)*Ni
    
    C2 = ((Ni+Ni_up)*(1-alpha2))/(Mx**(1-alpha2) - Mlim**(1-alpha2))
    C1 = C2*Mc**(alpha1 - alpha2)
    
    Ncorrs_up = []
    for c1, c2, mlim, ni in zip(C1,C2,Mlim, Ni + Ni_up):
        if mlim!=0:
            M = np.linspace(Ml,Mc)  
            Nlim1 = np.trapz(c1*M**(-alpha1), M)

            M = np.linspace(Mc,mlim)  
            Nlim2 = np.trapz(c2*M**(-alpha2), M)

            Ncorr = ni + Nlim1 + Nlim2
        else:
            Ncorr = 0
        Ncorrs_up.append(Ncorr)
    Ncorrs_up = np.array(Ncorrs_up)
    
    C2 = ((Ni-Ni_down)*(1-alpha2))/(Mx**(1-alpha2) - Mlim**(1-alpha2))
    C1 = C2*Mc**(alpha1 - alpha2)
    
    Ncorrs_down = []
    for c1, c2, mlim, ni in zip(C1,C2,Mlim, Ni - Ni_down):
        if mlim!=0:
            M = np.linspace(Ml,Mc)  
            Nlim1 = np.trapz(c1*M**(-alpha1), M)

            M = np.linspace(Mc,mlim)  
            Nlim2 = np.trapz(c2*M**(-alpha2), M)

            Ncorr = ni + Nlim1 + Nlim2
        else:
            Ncorr = 0
        Ncorrs_down.append(Ncorr)
    Ncorrs_down = np.array(Ncorrs_down)
    
    # Mean Mass
    h2 = (Mx**(1.-alpha2)-Mc**(1.-alpha2))/(1.-alpha2)
    h2 *= Mc**(alpha2-alpha1)
    h1 = (Mc**(1.-alpha1)-Ml**(1.-alpha1))/(1.-alpha1)

    C1 = 1/(h1+h2)
    C2 = C1*Mc**(alpha2-alpha1)

    Mean_M = []
    for c1, c2, mu in zip(C1,C2,Mx):
        M = np.linspace(Ml,Mc)  
        Mean_M1 = np.trapz(M*c1*M**(-alpha1), M)

        M = np.linspace(Mc,mu)  
        Mean_M2 = np.trapz(M*c2*M**(-alpha2), M)
        
        Mean_M.append(Mean_M1+Mean_M2)
      
    Mean_M = np.array(Mean_M)
    
    ages_ = np.array([ages[0]-0.1] + list(ages))
    
    delta_t = 10**ages_[1:]- 10**ages_[:-1]
    
    SFRs = (Mean_M*Ncorrs)/delta_t
    SFRs_err_up = abs((Mean_M*Ncorrs_up)/delta_t - SFRs)
    SFRs_err_down = abs((Mean_M*Ncorrs_down)/delta_t - SFRs)
    
    df_out['Ncorr'] = Ncorrs
    df_out['SFR'] = SFRs
    
    x = df_out['Log_age']
    y = SFRs*1e3
    
    fig, ax = plt.subplots(figsize=(12,8))
    ax.step(x,y,where='mid', color='blue')
    ax.errorbar(x,y,yerr=[SFRs_err_up*1e3, SFRs_err_down*1e3],
                fmt='o', color = 'red', markersize=0.5, capsize=2)
    
    #ax.step(x_,10*y_*1e3,where='mid')
    
    ax.set_xlabel(r'$\log(Age)~[yr]$')
    ax.set_ylabel(r'$SFR~[[10^{-3}]M_{\odot}.yr^{-1}]$')

    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.tick_params(which='both', width=3,direction="in", top = True,right = True,
                   bottom = True, left = True)
    ax.tick_params(which='major', length=15,direction="in")
    ax.tick_params(which='minor', length=8, color='black',direction="in")
    ax.set_yscale('log')
    fig.savefig(f'{out_dir}/plots/{region}_SFH_log.png', bbox_inches='tight')
    plt.close(fig)
    
    fig, ax = plt.subplots(figsize=(12,8))
    ax.step(x,y,where='mid', color='blue')
    ax.errorbar(x,y,yerr=[SFRs_err_up*1e3, SFRs_err_down*1e3],
                fmt='o', color = 'red', markersize=0.5, capsize=2)
    #ax.step(x_,y_*1e3,where='mid')
    

    ax.set_xlabel(r'$\log(Age)~[yr]$')
    ax.set_ylabel(r'$SFR~[[10^{-3}]M_{\odot}.yr^{-1}]$')

    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.tick_params(which='both', width=3,direction="in", top = True,right = True,
                   bottom = True, left = True)
    ax.tick_params(which='major', length=15,direction="in")
    ax.tick_params(which='minor', length=8, color='black',direction="in")
    fig.savefig(f'{out_dir}/plots/{region}_SFH.png', bbox_inches='tight')
    plt.close(fig)
    df_out.to_csv(f'{out_dir}/{region}_SFH.csv', index = None)

Sampling:  18% (2511/14000)
Sampling:  19% (2610/14000)
Sampling:  19% (2709/14000)
Sampling:  20% (2808/14000)
Sampling:  21% (2907/14000)
Sampling:  21% (3006/14000)
Sampling:  22% (3105/14000)
Sampling:  23% (3204/14000)
Sampling:  24% (3303/14000)
Sampling:  24% (3402/14000)
Sampling:  25% (3501/14000)
Sampling:  26% (3600/14000)
Sampling:  26% (3700/14000)
Sampling:  27% (3800/14000)
Sampling:  28% (3900/14000)
Sampling:  29% (4000/14000)
Sampling:  29% (4100/14000)
Sampling:  30% (4200/14000)
Sampling:  31% (4300/14000)
Sampling:  31% (4400/14000)
Sampling:  32% (4500/14000)
Sampling:  33% (4600/14000)
Sampling:  34% (4700/14000)
Sampling:  34% (4800/14000)
Sampling:  35% (4900/14000)
Sampling:  36% (5000/14000)
Sampling:  36% (5100/14000)
Sampling:  37% (5200/14000)
Sampling:  38% (5300/14000)
Sampling:  39% (5400/14000)
Sampling:  39% (5500/14000)
Sampling:  40% (5600/14000)
Sampling:  41% (5700/14000)
Sampling:  41% (5800/14000)
Sampling:  42% (5900/14000)
Sampling:  43% (6000

Completed!!!
Number of objects in the selected region: 15604
Selecting 5343 from 15604... (FW1 <= 25.55) (FW2 <= 24.50) (FW3 <= 24.13)
Starting Pij, Cij computation
	Parallel mode...
Finished                                
Elapsed time: 00:02:40


Building: found in cache, done.
Messages from stanc:
Warning in '/tmp/httpstan_kzhpznha/model_4xnllnqo.stan', line 10, column 16: A
    control flow statement inside function P depends on argument v. At
    '/tmp/httpstan_kzhpznha/model_4xnllnqo.stan', line 32, column 30 to
    column 31, the value of v depends on parameter(s): a.
Sampling:   0%
Sampling:   0% (1/14000)
Sampling:   0% (2/14000)
Sampling:   0% (3/14000)
Sampling:   0% (4/14000)
Sampling:   0% (5/14000)
Sampling:   0% (6/14000)
Sampling:   0% (7/14000)
Sampling:   0% (8/14000)
Sampling:   0% (9/14000)
Sampling:   0% (10/14000)
Sampling:   0% (11/14000)
Sampling:   0% (12/14000)
Sampling:   1% (111/14000)
Sampling:   2% (210/14000)
Sampling:   2% (309/14000)
Sampling:   3% (408/14000)
Sampling:   4% (507/14000)
Sampling:   4% (606/14000)
Sampling:   5% (705/14000)
Sampling:   6% (804/14000)
Sampling:   6% (903/14000)
Sampling:   7% (1002/14000)
Sampling:   8% (1101/14000)
Sampling:   9% (1200/14000)
Sampling:   9% (1300/1

Completed!!!
Number of objects in the selected region: 15300
Selecting 6036 from 15300... (FW1 <= 25.42) (FW2 <= 24.58) (FW3 <= 24.33)
Starting Pij, Cij computation
	Parallel mode...
Finished                                
Elapsed time: 00:25:57


Building: found in cache, done.
Messages from stanc:
Warning in '/tmp/httpstan_kzhpznha/model_4xnllnqo.stan', line 10, column 16: A
    control flow statement inside function P depends on argument v. At
    '/tmp/httpstan_kzhpznha/model_4xnllnqo.stan', line 32, column 30 to
    column 31, the value of v depends on parameter(s): a.
Sampling:   0%
Sampling:   0% (1/14000)
Sampling:   0% (2/14000)
Sampling:   0% (3/14000)
Sampling:   0% (4/14000)
Sampling:   0% (5/14000)
Sampling:   0% (6/14000)
Sampling:   0% (7/14000)
Sampling:   0% (8/14000)
Sampling:   0% (9/14000)
Sampling:   0% (10/14000)
Sampling:   0% (11/14000)
Sampling:   0% (12/14000)
Sampling:   1% (111/14000)
Sampling:   2% (210/14000)
Sampling:   2% (309/14000)
Sampling:   3% (408/14000)
Sampling:   4% (507/14000)
Sampling:   4% (606/14000)
Sampling:   5% (705/14000)
Sampling:   6% (804/14000)
Sampling:   6% (903/14000)
Sampling:   7% (1002/14000)
Sampling:   8% (1101/14000)
Sampling:   9% (1200/14000)
Sampling:   9% (1301/1

In [ ]:
sfh_obs = glob.glob("../SFH/ngc628/JWST_obs/*SFH.csv")
sfh_spatial = glob.glob("../SFH/ngc628/JWST_age/*spatial.csv")
sfh_sim = glob.glob("../SFH/ngc628/JWST_0/*SFH.csv")

sfh_obs = sorted(sfh_obs, key=lambda x: int(x.split("_")[-2]))
sfh_spatial = sorted(sfh_spatial, key=lambda x: int(x.split("_")[-2]))
sfh_sim = sorted(sfh_sim, key=lambda x: int(x.split("_")[-2]))

In [ ]:
df_bubbles = df_bubbles.sort_values('radius', ascending=False)

In [ ]:
p = 0
im_grid = np.arange(0,25).reshape(5,5)

#fig, axs = plt.subplots(5,5,figsize=(40,40), sharex=True, sharey=True)

for obs, spatial, sim in zip(sfh_obs, sfh_spatial, sfh_sim):
    fig, ax = plt.subplots(figsize=(12,10))
    obs_df = pd.read_csv(obs)
    N =  len(pd.read_csv(spatial)) 
    sim_df = pd.read_csv(sim)
    
    sim_Ncorr = sim_df['Ncorr']
    #print(y)
    corr = Ncorrs_/sim_Ncorr
    
    ai_10 = obs_df['p10']
    ai_10 /= ai_10.sum()
    
    ai = obs_df['p50']
    ai = ai/ai.sum()
    
    ai_90 = obs_df['p90']
    ai_90 /=ai_90.sum()
    
    Ni = ai*N
    
    fws = [f115w_m50_bubbles[p], f150w_m50_bubbles[p], f200w_m50_bubbles[p]]
    
    FW1 = fws[0] - AF115 - dismod
    FW2 = fws[1] - AF150 - dismod
    FW3 = fws[2] - AF200 - dismod
    
    Mx = np.array([i[1].max() for i in sfh.Iso])
    Mlim = []
    for i in sfh.Iso:
        j = i[1][(i[2]<=FW1) & (i[3]<=FW2) & (i[4]<=FW3)]
        if len(j)>0:
            Mlim.append(j.min())
        else:
            Mlim.append(0)
    Mlim = np.array(Mlim)
    
    C2 = (Ni*(1-alpha2))/(Mx**(1-alpha2) - Mlim**(1-alpha2))
    C1 = C2*Mc**(alpha1 - alpha2)
    
    # Ncorrs
    Ncorrs = []
    for c1, c2, mlim, ni in zip(C1,C2,Mlim, Ni):
        if mlim!=0:
            M = np.linspace(Ml,Mc)  
            Nlim1 = np.trapz(c1*M**(-alpha1), M)

            M = np.linspace(Mc,mlim)  
            Nlim2 = np.trapz(c2*M**(-alpha2), M)

            Ncorr = ni + Nlim1 + Nlim2
        else:
            Ncorr = 0
        Ncorrs.append(Ncorr)
    Ncorrs = np.array(Ncorrs)*corr
    
    # N errors
    Ni_up = ai_90*N
    Ni_down = ai_10*N
    
    C2 = ((Ni_up)*(1-alpha2))/(Mx**(1-alpha2) - Mlim**(1-alpha2))
    C1 = C2*Mc**(alpha1 - alpha2)
    
    Ncorrs_up = []
    for c1, c2, mlim, ni in zip(C1,C2,Mlim, Ni_up):
        if mlim!=0:
            M = np.linspace(Ml,Mc)  
            Nlim1 = np.trapz(c1*M**(-alpha1), M)

            M = np.linspace(Mc,mlim)  
            Nlim2 = np.trapz(c2*M**(-alpha2), M)

            Ncorr = ni + Nlim1 + Nlim2
        else:
            Ncorr = 0
        Ncorrs_up.append(Ncorr)
    Ncorrs_up = np.array(Ncorrs_up)*corr
    
    C2 = ((Ni_down)*(1-alpha2))/(Mx**(1-alpha2) - Mlim**(1-alpha2))
    C1 = C2*Mc**(alpha1 - alpha2)
    
    Ncorrs_down = []
    for c1, c2, mlim, ni in zip(C1,C2,Mlim, Ni_down):
        if mlim!=0:
            M = np.linspace(Ml,Mc)  
            Nlim1 = np.trapz(c1*M**(-alpha1), M)

            M = np.linspace(Mc,mlim)  
            Nlim2 = np.trapz(c2*M**(-alpha2), M)

            Ncorr = ni + Nlim1 + Nlim2
        else:
            Ncorr = 0
        Ncorrs_down.append(Ncorr)
    Ncorrs_down = np.array(Ncorrs_down)*corr
    
    
    
    # N poisson error 
    Ni_up_p = Ni + np.sqrt(Ni)
    Ni_down_p = Ni + np.sqrt(Ni)
    
    C2 = ((Ni_up)*(1-alpha2))/(Mx**(1-alpha2) - Mlim**(1-alpha2))
    C1 = C2*Mc**(alpha1 - alpha2)
    
    Ncorrs_up_p = []
    for c1, c2, mlim, ni in zip(C1,C2,Mlim, Ni_up_p):
        if mlim!=0:
            M = np.linspace(Ml,Mc)  
            Nlim1 = np.trapz(c1*M**(-alpha1), M)

            M = np.linspace(Mc,mlim)  
            Nlim2 = np.trapz(c2*M**(-alpha2), M)

            Ncorr = ni + Nlim1 + Nlim2
        else:
            Ncorr = 0
        Ncorrs_up_p.append(Ncorr)
    Ncorrs_up_p = np.array(Ncorrs_up_p)*corr
    
    C2 = ((Ni_down)*(1-alpha2))/(Mx**(1-alpha2) - Mlim**(1-alpha2))
    C1 = C2*Mc**(alpha1 - alpha2)
    
    Ncorrs_down_p = []
    for c1, c2, mlim, ni in zip(C1,C2,Mlim, Ni - Ni_down_p):
        if mlim!=0:
            M = np.linspace(Ml,Mc)  
            Nlim1 = np.trapz(c1*M**(-alpha1), M)

            M = np.linspace(Mc,mlim)  
            Nlim2 = np.trapz(c2*M**(-alpha2), M)

            Ncorr = ni + Nlim1 + Nlim2
        else:
            Ncorr = 0
        Ncorrs_down_p.append(Ncorr)
    Ncorrs_down_p = np.array(Ncorrs_down_p)*corr
    
    # Mean Mass
    h2 = (Mx**(1.-alpha2)-Mc**(1.-alpha2))/(1.-alpha2)
    h2 *= Mc**(alpha2-alpha1)
    h1 = (Mc**(1.-alpha1)-Ml**(1.-alpha1))/(1.-alpha1)

    C1 = 1/(h1+h2)
    C2 = C1*Mc**(alpha2-alpha1)

    Mean_M = []
    for c1, c2, mu in zip(C1,C2,Mx):
        M = np.linspace(Ml,Mc)  
        Mean_M1 = np.trapz(M*c1*M**(-alpha1), M)

        M = np.linspace(Mc,mu)  
        Mean_M2 = np.trapz(M*c2*M**(-alpha2), M)
        
        Mean_M.append(Mean_M1+Mean_M2)
      
    Mean_M = np.array(Mean_M)
    
    ages = obs_df['Log_age'].values
    ages_ = np.array([ages[0]-0.1] + list(ages))
    
    delta_t = 10**ages_[1:]- 10**ages_[:-1]
    
    SFRs = (Mean_M*Ncorrs)/delta_t
    SFRs_err_up = abs((Mean_M*Ncorrs_up)/delta_t- SFRs)
    SFRs_err_up_p = ((Mean_M*Ncorrs_up_p)/delta_t - SFRs)
    
    SFRs_err_up = np.sqrt(SFRs_err_up**2 +  SFRs_err_up_p**2)
    
    SFRs_err_down = abs((Mean_M*Ncorrs_down)/delta_t-SFRs)
    SFRs_err_down_p = abs((Mean_M*Ncorrs_down_p)/delta_t-SFRs)
    SFRs_err_down = np.sqrt( SFRs_err_down**2  +  SFRs_err_down_p**2)

    corr_mask = np.where(corr>100,np.nan,1)
    age_mask = np.where(ages<=9,1,np.nan)
    
    x = ages
    y = SFRs*1e3*corr_mask*age_mask
    
    SFR_max =np.nanmax(y)
    
    #ax = axs[*np.argwhere(im_grid==p)[0]]
    
    ax.step(x,y/SFR_max,where='mid', color='blue', label='SFR (Corrected)',
           lw=3)
    ax.errorbar(x,y/SFR_max,yerr=[(SFRs_err_up*1e3)/SFR_max, (SFRs_err_down*1e3)/SFR_max],
                fmt='o', color = 'red', markersize=0.5, capsize=2)
    
    #ax.step(x,obs_df['SFR']*1e3,where='mid', color='black',label='SFR')
    
    if True or p>=20:
        ax.set_xlabel(r'$\log(Age)~[yr]$')
    if True or (p + 5)%5 ==0:
        ax.set_ylabel('Normalized SFR')

    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.tick_params(which='both', width=3,direction="in", top = True,right = True,
                   bottom = True, left = True)
    ax.tick_params(which='major', length=15,direction="in")
    ax.tick_params(which='minor', length=8, color='black',direction="in")
    ax.legend(fontsize=20)
    text = f"Bubble {p}\n"+r" $SFR_{max}$: " + f"{np.round(SFR_max,2)} " + r'$\times10^{-3}[M_{\odot}.yr^{-1}]$'
    t1 = f'bubble_{p}'
    text += f"""\nR : {df_bubbles['radius'][t1]}" """
    ax.annotate(text, (6.85,1.), fontsize=25)
    ax.set_xlim(6.7,9)
    ax.set_ylim(0,1.4)
 
    fig.savefig(f"../SFH/ngc628/JWST_corr/Bubble_{p}.png", bbox_inches='tight')
    
    plt.close(fig)
    p+=1   

#plt.subplots_adjust(hspace=0, wspace=0)

In [ ]:
df_bubbles.index

In [ ]:
df_spatial = pd.read_csv(sfh_spatial[1])

In [ ]:
f560w = fits.open("../data/JWST/jw01783-o908_t016_miri_f560w/jw01783-o908_t016_miri_f560w_i2d.fits")

In [ ]:
ra = df_bubbles['ra']['bubble_0']
dec  = df_bubbles['dec']['bubble_0']
radius = df_bubbles['radius']['bubble_0']

In [ ]:
f560w_data = f560w[1].data
f560w_wcs = WCS(f560w[1].header)

coord = SkyCoord(ra=ra, dec=dec, unit='deg')
bubble_cutout = Cutout2D(f560w_data, position=coord,size=60*u.arcsec, wcs=f560w_wcs)

In [ ]:
crpix = bubble_cutout.data.shape[0]//2,bubble_cutout.data.shape[0]//2
crval =  bubble_cutout.wcs.pixel_to_world_values(*crpix)
wcs_new = bubble_cutout.wcs.deepcopy()

wcs_new.wcs.crval = crval
wcs_new.wcs.crpix = crpix

angle             = 90*np.pi/180

wcs_new.wcs.pc = np.array([[-np.sin(angle), np.cos(angle),],
                           [np.cos(angle),  np.sin(angle)]])

bubble_new,_ = reproject_interp(input_data=(bubble_cutout.data, bubble_cutout.wcs), output_projection=wcs_new)


bubble_cutout = Cutout2D(bubble_new, position=coord,size=30*u.arcsec, wcs=wcs_new)

In [ ]:
x = df_spatial['RA']
y = df_spatial['DEC']
c = df_spatial['Log_Age']
c_mask = np.where((c>6.7)&(c<=7),1,np.nan)


fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(projection=bubble_cutout.wcs)
norm = simple_norm(bubble_cutout.data, 'log',vmin=0., vmax=10, log_a=100)
ax.imshow(bubble_cutout.data, norm=norm, cmap='gray')

#ax.contour(bubble_cutout.data,levels = [1])

img = ax.scatter(x,y,c=c*c_mask, s=10, cmap='jet', vmin=6.8, vmax=8,
                transform = ax.get_transform("icrs"))

cb = plt.colorbar(img, ax=ax, shrink=0.8)

r = SphericalCircle((ra* u.deg, dec* u.deg), radius * u.arcsec,
                     edgecolor='red', facecolor='none',
                     transform=ax.get_transform('icrs'))

ax.add_patch(r)

add_scalebar(ax, 11.256286476305236/3600, label="""11.25" = 0.5kpc""", color="white",
             size_vertical = 2,borderpad = 0.25,
            fontproperties=fm.FontProperties(size=25, family='monospace'))

cb.set_label('log (Age)')
ax.invert_xaxis()

In [ ]:
r_young = []
r_old = []
r = []
bub_id = []
for i, spatial in enumerate(sfh_spatial):

    df_spatial = pd.read_csv(spatial)
    ra = df_bubbles['ra'][f'bubble_{i}']
    dec = df_bubbles['dec'][f'bubble_{i}']
    radius = df_bubbles['radius'][f'bubble_{i}']
    
    if i in [19]:
        continue
    
    df_spatial = df_spatial[(df_spatial['Log_Age']>6.7) & (df_spatial['Log_Age']<=7.5)]
    
    rs = angular_separation(ra*u.deg, dec*u.deg, df_spatial['RA'].values*u.deg, df_spatial['DEC'].values*u.deg).to(u.arcsec).value
    
    y,x = np.histogram(rs, bins = np.arange(0,radius, 0.1), density =True)
    x = 0.5*(x[1:] + x[:-1])
    cum_y = np.cumsum(y)*(x[1]-x[0])
    
    r_young.append(x[cum_y>0.75][0])
    r.append(radius)
    bub_id.append(i)
r_young = np.array(r_young)
r = np.array(r)

In [ ]:
fig, ax = plt.subplots(figsize=(7,4))
ax.scatter(r,r_young/r)

for x,y, i in  zip(r, r_young/r, bub_id):
    ax.annotate(str(i), (x,y+0.05))
    
ax.set_xlabel(r'$R_{Bubble}$')
ax.set_ylabel(r'$R_{Young}/R_{Bubble}$')

ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='both', width=2,direction="in", top = True,right = True,
               bottom = True, left = True)
ax.tick_params(which='major', length=7,direction="in")
ax.tick_params(which='minor', length=4, color='black',direction="in")

ax.set_ylim(0.4,1)

In [ ]:
np.tan(np.deg2rad(1/3600))*d

In [ ]:
500/44.41962285275099

In [ ]:
##### from astroML.correlation import two_point_angular

In [ ]:
bins = np.linspace(0.1/3600,20/3600,20)
bin_centers = 0.5*(bins[:-1] + bins[1:])

In [ ]:
df = df_spatial[(df_spatial['Log_Age']>6.7) & (df_spatial['Log_Age']<=7.5)]

df['r'] = 

In [ ]:
x = sim_df['Log_age']
y = sim_df['SFR']

fig, ax = plt.subplots(figsize=(12,7))
ax.step(x,y_n/1e-3, label = "Observed")
ax.step(x,y_corr/1e-3, label = "Corrected")

ax.set_xlabel(r'$\log(Age)~[yr]$')
ax.set_ylabel(r'$SFR~[[10^{-3}]M_{\odot}.yr^{-1}]$')

ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='both', width=3,direction="in", top = True,right = True,
               bottom = True, left = True)
ax.tick_params(which='major', length=15,direction="in")
ax.tick_params(which='minor', length=8, color='black',direction="in")
i=0
ax.set_title(f"F115W: {np.round(f115w_m50_bubbles[i],2)}, F150W: {np.round(f150w_m50_bubbles[i],2)}, F200W: {np.round(f200w_m50_bubbles[i],2)}")
ax.legend()

In [ ]:
sfh_obs_fs = glob(

In [ ]:
incomp_frac = y_/SFRs

In [ ]:
incomp_frac

In [ ]:
SFRs

In [ ]:
z = P_ij[-5]
x = df_out['Log_age']
plt.step(x,z, where='mid')

In [ ]:
delta_t

In [ ]:
x = df['mag_vega_F115W'] - df['mag_vega_F200W']
y = df['mag_vega_F200W'] + dismod
c = df['Log_Age']

fig, ax = plt.subplots(figsize=(10, 10))

img = ax.scatter(x,y,c=c,s=1, cmap='jet', label='_nolegend_')
cb = plt.colorbar(img, ax=ax)
ax.invert_yaxis()

ind = -12
ax.plot(sfh.Iso[ind][2]-sfh.Iso[ind][4], sfh.Iso[ind][4] + dismod, label = f'{ages[ind]}')
ax.legend()
ax.set_ylim(27,18)

In [ ]:
fs = glob.glob(f'../SFH/ngc628/reg_*_spatial.csv')

In [ ]:
dfs = []
for f in fs:
    df = pd.read_csv(f)
    dfs.append(df)

In [ ]:
df = pd.concat(dfs)

In [ ]:
probs = []
for n,i in enumerate(df['Prob']):
    try:
        probs.append(float(i[1:-1]))
    except:
        probs.append(np.nan)

df['Prob'] = probs

In [ ]:
df_filt = df[(df['Log_Age']>8.) & (df['Log_Age']<=9.) ]
x = df_filt['RA']
y = df_filt['DEC']
c = df_filt['Log_Age']
plt.figure(figsize=(10,10))
plt.scatter(x,y,c=c,s=0.1, cmap='jet')
plt.axis('equal')